In [1]:
import os
import re
import time
import pickle
import asyncio
import nest_asyncio
import pandas as pd
import numpy as np
import json
import logging
from tqdm import tqdm
from collections import defaultdict
from IPython.display import clear_output
from typing import List, Dict, Any, Optional
from sklearn.metrics.pairwise import cosine_similarity

from dotenv import load_dotenv
from langchain.docstore.document import Document
from langchain.chains import RetrievalQA, LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.embeddings import ClovaXEmbeddings
from langchain_community.chat_models import ChatClovaX
from pymilvus import connections, utility
from langchain_community.vectorstores.milvus import Milvus
from langchain.schema import BaseRetriever, Document
from sklearn.metrics.pairwise import cosine_similarity
from pydantic import BaseModel, Field

from langchain_elasticsearch import ElasticsearchRetriever
from elasticsearch import Elasticsearch, helpers
from langchain_elasticsearch import ElasticsearchStore
from langchain.retrievers import EnsembleRetriever

In [2]:
# 로깅 설정
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)

In [ ]:
# ISBNMergingRetriever



class ISBNMergingRetriever(BaseRetriever):
    """


    검색된 Document 리스트를 ISBN 기준으로 그룹화하고,


    같은 ISBN을 가진 문서들의 page_content를 병합하며,


    그룹 내에서 가장 정보가 많은 메타데이터를 대표로 사용합니다.
    """


    base_retriever: BaseRetriever


    def _extract_isbn(self, doc: Document) -> Optional[str]:


        try:


            for key in ["ISBN", "isbn"]:


                if key in doc.metadata:


                    isbn_val = doc.metadata[key]


                    return str(isbn_val).replace(".0", "").strip() if isbn_val else None


            inner_meta = doc.metadata.get("metadata", {})


            if isinstance(inner_meta, dict):


                for key in ["ISBN", "isbn"]:


                    if key in inner_meta:


                        isbn_val = inner_meta[key]


                        return (
                            str(isbn_val).replace(".0", "").strip()
                            if isbn_val
                            else None
                        )


                inner_inner_meta = inner_meta.get("metadata", {})


                if isinstance(inner_inner_meta, dict):


                    for key in ["ISBN", "isbn"]:


                        if key in inner_inner_meta:


                            isbn_val = inner_inner_meta[key]


                            return (
                                str(isbn_val).replace(".0", "").strip()
                                if isbn_val
                                else None
                            )


            source_meta = doc.metadata.get("_source", {}).get("metadata", {})


            if isinstance(source_meta, dict):


                for key in ["ISBN", "isbn"]:


                    if key in source_meta:


                        isbn_val = source_meta[key]


                        return (
                            str(isbn_val).replace(".0", "").strip()
                            if isbn_val
                            else None
                        )


        except Exception as e:


            logger.warning(f"[ISBN 추출 오류] Metadata: {doc.metadata}, Error: {e}")


        logger.debug(f"ISBN 추출 실패 - Metadata: {doc.metadata}")


        return None


    def _get_best_metadata(self, doc_list: List[Document]) -> Dict[str, Any]:


        best_meta = {}


        max_score = -1


        required_keys = {"title", "author", "ISBN"}


        for doc in doc_list:


            current_meta = doc.metadata


            score = 0


            keys_lower = {k.lower() for k in current_meta.keys()}


            required_keys_lower = {rk.lower() for rk in required_keys}


            score += sum(
                1
                for req_key in required_keys_lower
                if req_key in keys_lower
                and current_meta.get(
                    next((k for k in current_meta if k.lower() == req_key), None)
                )
            )


            score += len(current_meta) * 0.1


            if score > max_score:


                max_score = score


                best_meta = current_meta


        if best_meta:


            logger.debug(
                f"선택된 최적 메타데이터 (Score: {max_score:.2f}): {best_meta}"
            )


        else:


            logger.warning(
                "ISBN 그룹 내에서 유효한 메타데이터를 찾지 못함. 첫 번째 문서 메타데이터 사용 시도."
            )


            if doc_list:


                best_meta = doc_list[0].metadata


        return dict(best_meta)


    def _merge_documents_by_isbn(
        self, docs: List[Document], is_async=False
    ) -> List[Document]:


        grouped = defaultdict(list)


        merged_docs = []
        logger.info(

            f"[{'Async' if is_async else 'Sync'}] 병합 전 입력 문서 수: {len(docs)}"
        )


        for idx, doc in enumerate(docs):


            isbn = self._extract_isbn(doc)


            if isbn:


                grouped[isbn].append(doc)


            else:


                logger.warning(
                    f"[{'Async' if is_async else 'Sync'}] Doc {idx} ISBN 추출 실패 - Metadata: {doc.metadata}"
                )


        for isbn, doc_list in grouped.items():


            merged_meta = self._get_best_metadata(doc_list)


            combined_text = "\n\n---\n\n".join(
                d.page_content for d in doc_list if d.page_content
            ).strip()


            if combined_text:


                logger.debug(f"ISBN {isbn} 병합: 최종 사용될 메타데이터: {merged_meta}")


                merged_docs.append(
                    Document(page_content=combined_text, metadata=merged_meta)
                )


            else:


                logger.warning(
                    f"[{'Async' if is_async else 'Sync'}] ISBN {isbn} 병합 후 내용 없음. 제외됨. 사용된 메타데이터: {merged_meta}"
                )

        logger.info(

            f"[{'Async' if is_async else 'Sync'}] ISBN 병합 그룹 수: {len(grouped)}"
        )
        logger.info(

            f"[{'Async' if is_async else 'Sync'}] 병합 후 최종 문서 수: {len(merged_docs)}"
        )


        return merged_docs


    def _get_relevant_documents(self, query: str) -> List[Document]:


        try:


            docs = self.base_retriever.get_relevant_documents(query)


        except Exception as e:
            logger.error(

                f"기본 리트리버 동기 검색 오류: '{query}' → {e}", exc_info=True
            )


            docs = []


        logger.info(f"Sync 검색 결과 총 문서 수: {len(docs)}")


        return self._merge_documents_by_isbn(docs, is_async=False)


    async def _aget_relevant_documents(self, query: str) -> List[Document]:


        try:


            docs = await self.base_retriever.aget_relevant_documents(query)


        except Exception as e:
            logger.error(

                f"기본 리트리버 비동기 검색 오류: '{query}' → {e}", exc_info=True
            )


            docs = []


        logger.info(f"Async 검색 결과 총 문서 수: {len(docs)}")


        return self._merge_documents_by_isbn(docs, is_async=True)

##### Util Function

In [4]:
def is_similar_question(new_emb, prev_embeds, threshold=0.65):
    if not prev_embeds:
        return False
    sim_scores = cosine_similarity([new_emb], prev_embeds)[0]
    max_score = np.max(sim_scores)
    logger.info(
        f"[질문 유사도 판단] Max Similarity = {max_score:.3f} (Threshold = {threshold})"
    )
    return max_score > threshold


def extract_field(text, field_name):
    pattern = rf"^\s*{re.escape(field_name)}\s*[:：]\s*(.*?)\s*$"
    lines = text.splitlines()
    for line in lines:
        match = re.search(pattern, line, re.IGNORECASE)
        if match:
            return match.group(1).strip()
    return ""

##### 환경설정 & 임베딩

In [5]:
load_dotenv()

api_key = os.getenv("NCP_CLOVASTUDIO_API_KEY")
api_url = os.getenv("NCP_CLOVASTUDIO_API_URL", "https://clovastudio.stream.ntruss.com/")
milvus_host = os.getenv("MILVUS_HOST", "localhost")
milvus_port = os.getenv("MILVUS_PORT", "19530")
es_url = os.getenv("ELASTICSEARCH_URL", "http://localhost:9200")
es_index_name = "book_bm25_index_v2"

if not api_key:
    raise ValueError("NCP_CLOVASTUDIO_API_KEY 환경 변수가 설정되지 않았습니다.")
if not api_url:
    raise ValueError("NCP_CLOVASTUDIO_API_URL 환경 변수가 설정되지 않았습니다.")
if not es_url:
    raise ValueError("ELASTICSEARCH_URL 환경 변수가 설정되지 않았습니다.")

os.environ["NCP_CLOVASTUDIO_API_KEY"] = api_key
os.environ["NCP_CLOVASTUDIO_API_URL"] = api_url

try:
    ncp_embeddings = ClovaXEmbeddings(model="bge-m3")
    llm_clova = ChatClovaX(model="HCX-003", max_tokens=2048)
    logger.info("ClovaX 임베딩 및 Chat 모델 초기화 완료")
except Exception as e:
    logger.error(f"ClovaX 모델 초기화 실패: {e}")
    raise

2025-04-08 10:14:23,521 - INFO - ClovaX 임베딩 및 Chat 모델 초기화 완료


In [ ]:
embedding_file = r""
if os.path.exists(embedding_file):
    try:
        with open(embedding_file, "rb") as f:
            saved_data = pickle.load(f)
        all_text_embedding_pairs = [
            (v["text"], v["embedding"])
            for v in saved_data.values()
            if "text" in v and "embedding" in v
        ]
        all_metadata_list = [
            v["metadata"] for v in saved_data.values() if "metadata" in v
        ]
        if len(all_text_embedding_pairs) != len(all_metadata_list):
            logger.warning(
                f"로드된 텍스트/임베딩 쌍({len(all_text_embedding_pairs)})과 메타데이터({len(all_metadata_list)}) 개수 불일치."
            )
            min_len = min(len(all_text_embedding_pairs), len(all_metadata_list))
            all_text_embedding_pairs = all_text_embedding_pairs[:min_len]
            all_metadata_list = all_metadata_list[:min_len]
            logger.info(f"데이터를 {min_len}개로 조정하여 계속 진행.")
        logger.info(f"임베딩 데이터 로드 완료: {len(all_text_embedding_pairs)}개")
    except Exception as e:
        logger.error(f"임베딩 파일 로드 실패: {e}", exc_info=True)
        raise
else:
    raise FileNotFoundError(f"임베딩 파일을 찾을 수 없음: {embedding_file}")

metadata_mapping = {
    "ISBN": "ISBN",
    "페이지": "page",
    "가격": "price",
    "제목": "title",
    "부제": "subtitle",
    "저자": "author",
    "분류": "category",
    "저자소개": "author_intro",
    "책소개": "book_intro",
    "목차": "table_of_contents",
    "출판사리뷰": "publisher_review",
    "추천사": "recommendation",
    "발행자": "publisher",
    "표지": "book_cover",
}


def clean_metadata(meta: dict) -> dict:
    cleaned = {}
    target_keys = list(metadata_mapping.values())
    original_to_target = {
        k_orig: k_target for k_orig, k_target in metadata_mapping.items()
    }
    for target_key in target_keys:
        original_key = next(
            (k for k, v in original_to_target.items() if v == target_key), None
        )
        value = (
            meta.get(original_key)
            if original_key and original_key in meta
            else meta.get(target_key)
        )
        if pd.isna(value):
            cleaned[target_key] = 0 if target_key in ["page", "price"] else ""
        elif target_key == "ISBN":
            try:
                str_value = str(value).strip()
                cleaned[target_key] = (
                    str_value[:-2] if str_value.endswith(".0") else str_value
                )
            except Exception as e:
                logger.warning(f"ISBN 값 '{value}' 처리 중 오류 발생: {e}")
                cleaned[target_key] = str(value).strip()
        elif target_key == "subtitle":
            cleaned[target_key] = str(value)
        elif target_key in ["page", "price"]:
            try:
                cleaned[target_key] = int(float(value))
            except (ValueError, TypeError):
                logger.warning(
                    f"'{target_key}' 값 '{value}' 정수 변환 실패. 0으로 설정."
                )
                cleaned[target_key] = 0
        else:
            cleaned[target_key] = str(value)
    return cleaned


documents = []
for i, (pair, meta) in enumerate(zip(all_text_embedding_pairs, all_metadata_list)):
    try:
        cleaned_meta = clean_metadata(meta)
        documents.append(Document(page_content=pair[0], metadata=cleaned_meta))
    except Exception as e:
        logger.error(
            f"{i}번째 데이터 처리 중 오류 발생: {e}. 메타데이터: {meta}", exc_info=True
        )
logger.info(f"총 {len(documents)}개의 Document 객체 생성 완료.")

texts = [doc.page_content for doc in documents]
embeds = [pair[1] for pair in all_text_embedding_pairs[: len(documents)]]
metadatas = [doc.metadata for doc in documents]

2025-04-08 10:14:46,786 - INFO - 임베딩 데이터 로드 완료: 116218개
2025-04-08 10:14:50,950 - INFO - 총 116218개의 Document 객체 생성 완료.


In [7]:
# Milvus Vector DB
collection_name = "book_rag_db_v_no_sq"
temp_conn_alias = "utility_check_conn"
try:
    connections.connect(alias=temp_conn_alias, host=milvus_host, port=milvus_port)
    logger.info(
        f"Milvus 유틸리티 함수용 임시 연결 설정 완료 (alias: {temp_conn_alias})."
    )
    if utility.has_collection(collection_name, using=temp_conn_alias):
        logger.warning(f"기존 Milvus 컬렉션 '{collection_name}'을 삭제.")
        utility.drop_collection(collection_name, using=temp_conn_alias)
    else:
        logger.info(
            f"Milvus 컬렉션 '{collection_name}'이(가) 존재하지 않음. 새로 생성."
        )
except Exception as e:
    logger.error(f"Milvus 유틸리티 함수 실행 중 오류 발생: {e}", exc_info=True)
finally:
    try:
        if connections.get_connection_addr(temp_conn_alias):
            connections.disconnect(temp_conn_alias)
            logger.info(
                f"Milvus 유틸리티 함수용 임시 연결 해제 완료 (alias: {temp_conn_alias})."
            )
    except Exception as disconnect_e:
        logger.warning(f"Milvus 임시 연결 해제 중 오류 (무시 가능): {disconnect_e}")

try:
    vectorstore = Milvus(
        embedding_function=ncp_embeddings,
        collection_name=collection_name,
        connection_args={"host": milvus_host, "port": milvus_port},
        auto_id=True,
    )
    logger.info(
        f"Langchain Milvus 인스턴스 초기화 및 연결 설정 완료 (컬렉션: '{collection_name}')."
    )
    original_embed_documents = ClovaXEmbeddings.embed_documents

    def precomputed_embed_documents(cls, input_texts: List[str]) -> List[List[float]]:
        if len(input_texts) == len(texts) and all(
            t1 == t2 for t1, t2 in zip(input_texts, texts)
        ):
            logger.info(f"사전 계산된 임베딩 사용: {len(embeds)}개")
            return embeds
        else:
            logger.warning(
                "입력 텍스트가 사전 계산된 데이터와 불일치. ClovaX API를 통해 임베딩 수행."
            )
            return original_embed_documents.__func__(cls, input_texts)

    ClovaXEmbeddings.embed_documents = classmethod(precomputed_embed_documents)
    vectorstore.add_texts(texts=texts, metadatas=metadatas)
    logger.info(f"Milvus에 {len(texts)}개의 텍스트와 임베딩 추가 완료.")
    ClovaXEmbeddings.embed_documents = original_embed_documents
    logger.info("ClovaXEmbeddings.embed_documents 메소드 원상 복구 완료.")
except Exception as e:
    logger.error(f"Milvus 데이터 추가 중 오류 발생: {e}", exc_info=True)
    if (
        "original_embed_documents" in locals()
        and hasattr(ClovaXEmbeddings, "embed_documents")
        and ClovaXEmbeddings.embed_documents != original_embed_documents
    ):
        ClovaXEmbeddings.embed_documents = original_embed_documents
        logger.info(
            "오류 발생 후 ClovaXEmbeddings.embed_documents 메소드 원상 복구 시도 완료."
        )
    raise

2025-04-08 10:15:22,684 - INFO - Milvus 유틸리티 함수용 임시 연결 설정 완료 (alias: utility_check_conn).
2025-04-08 10:15:22,690 - WARNING - 기존 Milvus 컬렉션 'book_rag_db_v_no_sq'을 삭제.
2025-04-08 10:15:22,833 - INFO - Milvus 유틸리티 함수용 임시 연결 해제 완료 (alias: utility_check_conn).
C:\Users\computer\AppData\Local\Temp\ipykernel_30692\2085374808.py:29: LangChainDeprecationWarning: The class `Milvus` was deprecated in LangChain 0.2.0 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-milvus package and should be used instead. To use it run `pip install -U :class:`~langchain-milvus` and import as `from :class:`~langchain_milvus import MilvusVectorStore``.
  vectorstore = Milvus(
2025-04-08 10:15:22,888 - INFO - Langchain Milvus 인스턴스 초기화 및 연결 설정 완료 (컬렉션: 'book_rag_db_v_no_sq').
2025-04-08 10:15:22,910 - INFO - 사전 계산된 임베딩 사용: 116218개
2025-04-08 10:17:00,445 - INFO - Milvus에 116218개의 텍스트와 임베딩 추가 완료.
2025-04-08 10:17:00,571 - INFO - ClovaXEmbeddings.embed_documents 메소드 원상 복구 완

In [8]:
# Elasticsearch store 설정
try:
    logger.info(f"Elasticsearch 연결 시도 중: {es_url}...")
    es_client = Elasticsearch(hosts=[es_url], request_timeout=120)
    if not es_client.ping():
        raise ConnectionError("Elasticsearch 연결 실패. 서버 상태 및 URL 확인 필요.")
    logger.info("Elasticsearch 연결 성공")
    if es_client.indices.exists(index=es_index_name):
        logger.warning(f"기존 Elasticsearch 인덱스 '{es_index_name}' 삭제 중...")
        es_client.indices.delete(index=es_index_name, ignore=[400, 404])
        logger.info(f"기존 Elasticsearch 인덱스 '{es_index_name}' 삭제 완료.")
    else:
        logger.info(
            f"Elasticsearch 인덱스 '{es_index_name}' 존재하지 않음. 새로 생성됩니다."
        )
    es_store = ElasticsearchStore(
        index_name=es_index_name,
        es_connection=es_client,
        strategy=ElasticsearchStore.ExactRetrievalStrategy(),
    )
    logger.info(
        f"Langchain ElasticsearchStore 인스턴스 초기화 완료 (인덱스: '{es_index_name}', BM25 검색 전용)."
    )
    logger.info(f"Elasticsearch에 문서 인덱싱 시작 (총 {len(texts)}개)...")
    actions = [
        {
            "_index": es_index_name,
            "_source": {"text": texts[i], "metadata": metadatas[i]},
        }
        for i in range(len(texts))
    ]
    indexed_count, errors = helpers.bulk(es_client, actions, raise_on_error=False)
    logger.info(f"Elasticsearch 벌크 인덱싱 시도 완료: {indexed_count} 문서 성공.")
    if errors:
        logger.error(
            f"Elasticsearch 벌크 인덱싱 중 오류 발생: {len(errors)}개 문서 실패."
        )
        for i, error in enumerate(errors[:5]):
            logger.error(f"  실패 {i+1}: {error}")
    es_client.indices.refresh(index=es_index_name)
    logger.info(f"Elasticsearch 인덱스 '{es_index_name}' 새로고침 완료.")
except ConnectionError as ce:
    logger.error(f"Elasticsearch 연결 실패: {ce}")
    raise
except Exception as e:
    logger.error(
        f"Elasticsearch 설정 또는 데이터 추가 중 오류 발생: {e}", exc_info=True
    )
    raise

dense_retriever = vectorstore.as_retriever(search_kwargs={"k": 5})
logger.info("Dense retriever (Milvus 벡터 검색) 설정 완료 (k=5).")
try:
    if "es_client" not in locals() or not es_client.ping():
        raise ConnectionError(
            "Elasticsearch client가 준비되지 않았거나 연결할 수 없습니다."
        )
    sparse_retriever = ElasticsearchRetriever(
        es_client=es_client,
        index_name=es_index_name,
        body_func=lambda query: {
            "size": 5,
            "query": {"match": {"text": {"query": query}}},
        },
        content_field="text",
        metadata_field="metadata",
    )
    logger.info(
        "Sparse retriever (Elasticsearch BM25) 설정 완료 (ElasticsearchRetriever 직접 사용, metadata_field 지정)."
    )
except ConnectionError as ce:
    logger.error(f"Elasticsearch 클라이언트 오류: {ce}")
    raise
except Exception as e:
    logger.error(f"ElasticsearchRetriever 직접 설정 중 오류 발생: {e}", exc_info=True)
    raise

hybrid_retriever = EnsembleRetriever(
    retrievers=[sparse_retriever, dense_retriever], weights=[0.5, 0.5], c=60
)
logger.info(
    "Hybrid ensemble retriever 설정 완료 (Direct ES BM25: 0.5, Milvus Vector: 0.5)."
)
merged_hybrid_retriever = ISBNMergingRetriever(base_retriever=hybrid_retriever)
logger.info("ISBN Merging Retriever 설정 완료 (Hybrid 결과 기반).")
dpr_qa_chain = RetrievalQA.from_chain_type(
    llm=llm_clova, retriever=merged_hybrid_retriever, return_source_documents=True
)
logger.info("RetrievalQA 체인 설정 완료 (Merged Hybrid Retriever 사용).")

MIN_INFO_LENGTH = 10
previous_additional_question_embeddings = []

2025-04-08 10:17:10,105 - INFO - Elasticsearch 연결 시도 중: http://localhost:9200...
2025-04-08 10:17:36,934 - INFO - HEAD http://localhost:9200/ [status:200 duration:26.223s]
2025-04-08 10:17:36,942 - INFO - Elasticsearch 연결 성공
2025-04-08 10:17:37,612 - INFO - HEAD http://localhost:9200/book_bm25_index_v2 [status:200 duration:0.648s]
2025-04-08 10:17:37,613 - WARNING - 기존 Elasticsearch 인덱스 'book_bm25_index_v2' 삭제 중...
C:\Users\computer\AppData\Local\Temp\ipykernel_30692\3537059587.py:10: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es_client.indices.delete(index=es_index_name, ignore=[400, 404])
2025-04-08 10:17:44,152 - INFO - DELETE http://localhost:9200/book_bm25_index_v2 [status:200 duration:2.357s]
2025-04-08 10:17:44,155 - INFO - 기존 Elasticsearch 인덱스 'book_bm25_index_v2' 삭제 완료.
2025-04-08 10:17:44,161 - INFO - Langchain ElasticsearchStore 인스턴스 초기화 완료 (인덱스: 'book_bm25_index_v2', BM25 검색 전용).
2025-04-08 10:17:4

In [ ]:
# 1. 사용자 선호도 추출 프롬프트 (원본)

extract_pref_prompt_v2 = PromptTemplate(
    input_variables=["text"],
    template="""
다음 사용자 발화에서 사용자의 선호도 및 책에 대한 요구사항을 아래 JSON 형식으로 추출해라. 각 항목은 관련된 정보가 명확할 때만 명확한 항목에 포함(예: 소설 -> category)하고, 없다면 빈 리스트 [] 또는 빈 문자열 ""로 남겨라. 여러 개가 추출될 수 있는 항목은 리스트로 추출하라.
사용자 입력에서 모호한 정보는 implicit info로 포함해라.
존재하지 않는 사용자 선호도 정보는 임의로 생성하지 마라.

입력: {{ text }}

출력 형식 (JSON, 다른 설명 없이 JSON만 출력):
{
    "title": [<!-- 추출된 책 제목 -->],
    "author": [<!-- 추출된 책 저자 -->],
    "category": [<!-- 추출된 책 분류/장르 (예: 소설, 에세이, 기술 등) -->],
    "author_intro": [<!-- 저자 특성 언급/요구사항 -->],
    "book_intro": [<!-- 줄거리 관련 언급/요구사항 -->],
    "table_of_contents": [<!-- 세부적인 키워드 언급/요구사항 -->],
    "purpose": [<!-- 사용자의 독서 목적/이유 (예: 재미, 학습, 시간 때우기, 기분 등) -->],
    "implicit info": [<!-- 추천해야 할 책에 대한 암시적 정보/특징/분위기 (예: 밝은 분위기, 특정 상황에 어울리는 책, 최신 기술 동향 등) -->]
}
""",
    template_format="jinja2",
)


# 2. 선호도 통합 프롬프트 (원본)



consolidate_pref_prompt = PromptTemplate(
    input_variables=["existing_preferences", "new_preferences"],
    template="""
기존에 수집된 사용자 선호도 정보와 새로 추출된 선호도 정보가 주어졌다. 두 정보를 지능적으로 통합하여 중복을 제거하고 관련 내용을 요약/결합하여 최종 선호도 목록을 생성해라.

[기존 선호도]
{{ existing_preferences }}

[새로운 선호도]
{{ new_preferences }}

[통합된 최종 선호도 목록]
(아래 목록 형태로만 출력, 각 항목은 문자열 리스트)
- 항목1: ["통합 내용1", "통합 내용2"]
- 항목2: ["통합 내용3"]
...
""",
    template_format="jinja2",
)



# 3. Decision Prompt (원본)



decision_prompt_template = PromptTemplate(
    input_variables=["history", "query", "preferences", "role_instructions"],
    template="""
[대화 맥락]
사용자 대화 내역:
{{ history }}
사용자의 최신 질문: "{{ query }}"
수집된 사용자 선호도:
{{ preferences }}

[역할 및 목표]
{{ role_instructions }}

현재 대화 상황, 질문, 수집된 선호도를 분석하여 아래 두 가지 행동 중 하나만 결정하고 필요한 정보를 생성해라.
- "추천": 사용자가 명시적으로 추천을 요청했거나, 사용자의 선호도 정보(예: 카테고리, 저자, 목적, 책 줄거리, 사용자 수준, 분위기 등)를 반드시 3개 이상 수집했을 때 추천.
- "추가 질문": 정보가 부족하거나 모호할 때, 더 구체적인 선호도 정보를 얻기 위한 추가 질문을 생성.

[출력 형식] (반드시 아래 형식만 정확히 따를 것)
행동: <추천 또는 추가 질문>
추가 질문: <"추가 질문" 행동일 경우 구체적인 질문 생성, "추천"일 경우 빈 문자열>
""",
    template_format="jinja2",
)



# 4. Final Query Generation Prompt



literature_final_query_template = PromptTemplate(
    template="""
**명령:** 아래 제공된 모든 정보를 **반드시 종합적으로 분석**하여, 문학 도서 검색에 최적화된 **핵심 키워드 중심의 최종 검색 쿼리**를 **단 한 문장**으로 생성하라.

**입력 정보:**
[대화 요약]
{{ history }}
[사용자 요청]
{{ query }}
[페르소나 정보]
{{ persona_info }} (감성, 분위기, 문체, 작가 스타일 등 문학적 요소 강조)
[사용자 선호도 요약]
{{ preferences }}

**쿼리 생성 지침:**
1.  **정보 통합:** 모든 입력 정보(대화 맥락, 요청, 페르소나, 선호도)를 누락 없이 반영하라.
2.  **키워드 중심:** 검색 시스템이 효과적으로 인식할 수 있는 명사, 핵심 형용사 위주로 구성하라.
3.  **문학적 뉘앙스:** 페르소나 정보와 선호도의 감성, 분위기, 문체 관련 내용을 키워드에 포함시키되, 검색 효율성을 해치지 않도록 명료하게 표현하라.
4.  **단일 문장:** 최종 결과는 오직 검색 쿼리 한 문장이어야 한다.

**출력 (오직 최종 검색 쿼리 한 문장, 다른 설명 절대 금지):**
""",
    input_variables=["history", "query", "persona_info", "preferences"],
    template_format="jinja2",
)



science_final_query_template = PromptTemplate(
    template="""
**명령:** 아래 제공된 모든 정보를 **반드시 종합적으로 분석**하여, 과학/기술 도서 검색에 최적화된 **정확하고 구체적인 키워드 중심의 최종 검색 쿼리**를 **단 한 문장**으로 생성하라.

**입력 정보:**
[대화 요약]
{{ history }}
[사용자 요청]
{{ query }}
[페르소나 정보]
정확하고 논리적인 분석, 최신 기술 동향, 전문 지식, 데이터 기반 접근
[사용자 선호도 요약]
{{ preferences }}

**쿼리 생성 지침:**
1.  **정보 통합:** 모든 입력 정보(대화 맥락, 요청, 페르소나, 선호도)를 누락 없이 반영하라.
2.  **키워드 중심:** 검색 시스템이 효과적으로 인식할 수 있는 전문 용어, 기술 명칭, 핵심 개념 위주로 구성하라.
3.  **정확성/구체성:** 페르소나 정보와 선호도의 기술 분야, 난이도, 최신성 요구 등을 명확한 키워드로 반영하라.
4.  **단일 문장:** 최종 결과는 오직 검색 쿼리 한 문장이어야 한다.

**출력 (오직 최종 검색 쿼리 한 문장, 다른 설명 절대 금지):**
""",
    input_variables=["history", "query", "persona_info", "preferences"],
    template_format="jinja2",
)

general_final_query_template = PromptTemplate(
    template="""
**명령:** 아래 제공된 모든 정보를 **반드시 종합적으로 분석**하여, 범용 도서 검색에 가장 유용한 **핵심 키워드 중심의 최종 검색 쿼리**를 **단 한 문장**으로 생성하라.

**입력 정보:**
[대화 요약]
{{ history }}
[사용자 요청]
{{ query }}
[페르소나 정보]
{{ persona_info }} (친절, 균형잡힌 정보, 다양한 분야 포괄)
[사용자 선호도 요약]
{{ preferences }}

**쿼리 생성 지침:**
1.  **정보 통합:** 모든 입력 정보(대화 맥락, 요청, 페르소나, 선호도)를 누락 없이 반영하라.
2.  **키워드 중심:** 검색 시스템이 효과적으로 인식할 수 있는 명확한 명사, 핵심 요구사항 키워드 위주로 구성하라.
3.  **균형과 명료성:** 다양한 선호도를 반영하되, 가장 핵심적인 요구사항을 명확한 키워드로 표현하여 검색 효율성을 높여라.
4.  **단일 문장:** 최종 결과는 오직 검색 쿼리 한 문장이어야 한다.

**출력 (오직 최종 검색 쿼리 한 문장, 다른 설명 절대 금지):**
""",
    input_variables=["history", "query", "persona_info", "preferences"],
    template_format="jinja2",
)



# 5. Refine Prompt

literature_refine_template = PromptTemplate(
    input_variables=["query"],
    template="""
**명령:** 주어진 [원본 검색 쿼리]를 아래 지침에 따라 **문학적 감성을 반영**하면서도 **검색에 용이한 간결한 단일 문장**으로 정제하라.

**정제 지침:**
1.  **유사성 표현 처리:** "~~비슷한", "~ 같은", "~ 분위기의" 등의 표현이 있으면, 해당 대상의 **문학적 특징(장르, 문체, 감정선, 시대 등)을 분석하여 명시적인 키워드로 확장**하라. 단, 원본의 책 제목이나 저자 이름은 직접 포함하지 마라.
    *   예시 (입력: '데미안' 같은 성장 소설) -> 정제: 내면 성찰을 다루는 철학적 성장 소설 추천
2.  **핵심 의도 유지:** 원본 쿼리의 핵심 검색 의도는 반드시 보존해야 한다.
3.  **간결성 및 명확성:** 불필요한 수식어나 설명은 제거하고, 검색 키워드가 될 수 있는 명료한 단어 위주로 구성하라.
4.  **문학적 표현:** 서정적이거나 감성적인 표현을 사용하되, 검색 시스템이 이해할 수 있는 수준을 유지하라.

[원본 검색 쿼리]
{{ query }}

**출력 (오직 정제된 단일 검색 쿼리 문장, 다른 설명 절대 금지):**
[정제된 검색 쿼리]
""",
    template_format="jinja2",
)

science_refine_template = PromptTemplate(
    input_variables=["query"],
    template="""
**명령:** 주어진 [원본 검색 쿼리]를 아래 지침에 따라 **객관적이고 논리적인 표현**을 사용하여 **정확하고 간결한 단일 문장**으로 정제하라.

**정제 지침:**
1.  **유사성 표현 처리:** "~~비슷한", "~ 같은" 등의 표현이 있으면, 해당 대상의 **기술적 특징(핵심 기술, 분야, 방법론, 난이도 등)을 분석하여 명시적인 키워드로 확장**하라. 단, 원본의 책 제목이나 저자 이름은 직접 포함하지 마라.
    *   예시 (입력: '핸즈온 머신러닝' 같은 책) -> 정제: 사이킷런과 텐서플로우 예제 중심의 실습형 머신러닝 입문서 추천
2.  **핵심 의도 유지:** 원본 쿼리의 핵심 검색 의도는 반드시 보존해야 한다.
3.  **간결성 및 정확성:** 불필요한 표현은 제거하고, 정확한 기술 용어와 핵심 개념 위주로 구성하라.
4.  **객관적 표현:** 주관적이거나 모호한 표현 대신, 명확하고 객관적인 과학/기술 용어를 사용하라.

[원본 검색 쿼리]
{{ query }}

**출력 (오직 정제된 단일 검색 쿼리 문장, 다른 설명 절대 금지):**
[정제된 검색 쿼리]
""",
    template_format="jinja2",
)

general_refine_template = PromptTemplate(
    input_variables=["query"],
    template="""
**명령:** 주어진 [원본 검색 쿼리]를 아래 지침에 따라 **사용자 의도를 명확히 반영**하여 **친절하고 이해하기 쉬운 간결한 단일 문장**으로 정제하라.

**정제 지침:**
1.  **유사성 표현 처리:** "~~비슷한", "~ 같은", "~ 분위기의" 등의 표현이 있으면, 해당 대상의 **주요 특징(장르, 핵심 소재, 스타일, 목적 등)을 분석하여 명시적인 키워드로 확장**하라. 단, 원본의 책 제목이나 저자 이름은 직접 포함하지 마라.
    *   예시 (입력: '나미야 잡화점의 기적' 같은 힐링 소설) -> 정제: 따뜻한 위로와 감동을 주는 일본 힐링 소설 추천
2.  **핵심 의도 유지:** 원본 쿼리의 핵심 검색 의도는 반드시 보존해야 한다.
3.  **간결성 및 명확성:** 불필요한 수식어나 모호한 표현은 제거하고, 사용자의 요구사항을 명확히 나타내는 단어 위주로 구성하라.
4.  **친절한 표현:** 딱딱하거나 전문적이지 않은, 일반 사용자가 이해하기 쉬운 표현을 사용하라.

[원본 검색 쿼리]
{{ query }}

**출력 (오직 정제된 단일 검색 쿼리 문장, 다른 설명 절대 금지):**
[정제된 검색 쿼리]
""",
    template_format="jinja2",
)



# 6. Query Expansion Prompt

literature_expansion_template = PromptTemplate(
    input_variables=["query"],
    template="""
**명령:** 주어진 [원본 검색 쿼리]의 **핵심 의도를 반드시 유지**하면서, **문학적 감성과 관련된 다양한 측면(유사 장르, 다른 시대, 관련 주제, 다른 작가 등)을 탐색**할 수 있는 **관련성 높은 확장 검색 쿼리 3개**를 생성하라.

**확장 지침:**
1.  **핵심 의도 유지:** 확장 쿼리는 원본 쿼리의 주제나 분위기에서 크게 벗어나서는 안 된다.
2.  **문학적 다양성 추구:** 원본 쿼리와 관련되면서도 다른 각도(예: 다른 감정선, 문체, 배경)에서 탐색할 수 있도록 다양하게 생성하라.
3.  **관련성:** 생성된 쿼리는 원본 쿼리와 명확한 관련성을 가져야 한다.
4.  **형식 엄수:** **번호(1., 2., 3.)가 매겨진 목록 형식**으로, 각 줄에 확장 쿼리 하나씩만 출력하라. 다른 설명은 절대 포함하지 마라.

[원본 검색 쿼리]
{{ query }}

**출력 (오직 번호 매겨진 확장 쿼리 목록 3개, 다른 설명 절대 금지):**
[확장된 검색 쿼리 목록]
""",
    template_format="jinja2",
)

science_expansion_template = PromptTemplate(
    input_variables=["query"],
    template="""
**명령:** 주어진 [원본 검색 쿼리]의 **핵심 의도를 반드시 유지**하면서, **과학/기술 관련 하위 주제, 관련 기술, 응용 분야, 다른 접근법 등을 탐색**할 수 있는 **관련성 높은 확장 검색 쿼리 3개**를 생성하라.

**확장 지침:**
1.  **핵심 의도 유지:** 확장 쿼리는 원본 쿼리의 기술 분야나 주제에서 크게 벗어나서는 안 된다.
2.  **기술적 다양성 추구:** 원본 쿼리와 관련되면서도 다른 각도(예: 심화 이론, 실용적 적용, 다른 프로그래밍 언어, 최신 연구)에서 탐색할 수 있도록 다양하게 생성하라.
3.  **관련성:** 생성된 쿼리는 원본 쿼리와 명확한 기술적/논리적 관련성을 가져야 한다.
4.  **형식 엄수:** **번호(1., 2., 3.)가 매겨진 목록 형식**으로, 각 줄에 확장 쿼리 하나씩만 출력하라. 다른 설명은 절대 포함하지 마라.

[원본 검색 쿼리]
{{ query }}

**출력 (오직 번호 매겨진 확장 쿼리 목록 3개, 다른 설명 절대 금지):**
[확장된 검색 쿼리 목록]
""",
    template_format="jinja2",
)

general_expansion_template = PromptTemplate(
    input_variables=["query"],
    template="""
**명령:** 주어진 [원본 검색 쿼리]의 **핵심 의도를 반드시 유지**하면서, **관련 주제, 다른 관점, 사용자의 잠재적 관심사 등 다양한 분야를 탐색**할 수 있는 **관련성 높은 확장 검색 쿼리 3개**를 생성하라.

**확장 지침:**
1.  **핵심 의도 유지:** 확장 쿼리는 원본 쿼리의 핵심 주제나 요구사항에서 크게 벗어나서는 안 된다.
2.  **주제 다양성 추구:** 원본 쿼리와 관련되면서도 다른 각도(예: 유사하지만 다른 장르, 관련 인물 이야기, 사회적 배경)에서 탐색할 수 있도록 다양하게 생성하라.
3.  **관련성:** 생성된 쿼리는 원본 쿼리와 명확한 관련성을 가져야 한다.
4.  **형식 엄수:** **번호(1., 2., 3.)가 매겨진 목록 형식**으로, 각 줄에 확장 쿼리 하나씩만 출력하라. 다른 설명은 절대 포함하지 마라.

[원본 검색 쿼리]
{{ query }}

**출력 (오직 번호 매겨진 확장 쿼리 목록 3개, 다른 설명 절대 금지):**
[확장된 검색 쿼리 목록]
""",
    template_format="jinja2",
)



# 7. Re_ranking Prompt (원본)
re_ranking_prompt = PromptTemplate(
    input_variables=["query", "documents"],
    template="""
사용자의 검색 쿼리는 다음과 같습니다: "{{ query }}"
다음은 검색된 도서 목록입니다 (내용은 일부만 표시됨):
{% for doc in documents %}
{{ loop.index }}. 제목: {{ doc.metadata.get('title', '제목 없음') }}, 저자: {{ doc.metadata.get('author', '저자 없음') }}, 내용 일부: {{ doc.page_content | truncate(200) }}
{% endfor %}

위 검색 결과를 사용자의 검색 쿼리 "{{ query }}"와의 관련성 및 문서 내용의 충실도를 종합적으로 고려하여, 가장 관련성이 높은 도서를 목록의 맨 위로 배치하고, 결과를 도서 제목과 저자만 포함하는 다음 형식으로 출력하라.

[출력 형식 예시]
1. 제목: <가장 관련성 높은 책 제목>, 저자: <저자 이름>
2. 제목: <두 번째 관련성 높은 책 제목>, 저자: <저자 이름>
...
[리랭킹된 도서 목록]
""",
    template_format="jinja2",
)

# 8. HyDE Generation Prompt (원본)
hyde_generation_prompt = PromptTemplate(
    input_variables=["query"],
    template="""
다음 검색 쿼리에 완벽하게 부합하는 **이상적인 가상의 책**을 추천하고, 이를 기반으로 **간결한 요약(2-3 문장)**을 생성해라. 이 요약은 해당 쿼리로 책을 찾는 사용자가 가장 만족할 만한 내용을 담아야 한다. 오직 생성된 요약 텍스트만 출력하라.

[검색 쿼리]
{{ query }}

[가상의 책 요약]
""",
    template_format="jinja2",
)

# 9. Persona Role Instructions (원본)
literature_role = "너는 감성적이고 문학적인 도서 추천 챗봇이다. 사용자의 감정과 취향을 깊이 이해하고 공감하는 말투로 문학적인 표현을 사용하여 책을 추천해라."
science_role = "너는 정확하고 논리적인 과학/기술 도서 추천 챗봇이다. 사용자의 지식 수준과 관심 분야를 파악하고, 최신 정보와 기술 동향을 반영하여 체계적으로 책을 추천해라."
general_role = "너는 친절하고 신뢰할 수 있는 범용 도서 추천 챗봇이다. 친근한 말투로 다양한 분야의 책에 대해 균형 잡힌 시각으로 정보를 제공하고, 사용자의 요구사항에 맞춰 명확하고 이해하기 쉽게 책을 추천해라."

# 10. Hyde Keyword Prompt (원본)
hyde_keyword_prompt = PromptTemplate(
    input_variables=["hyde_summary"],
    template="""
다음은 사용자의 질문에 이상적으로 부합하는 가상의 책 요약입니다:
"{{ hyde_summary }}"

이 요약 내용에서 **핵심 키워드 5개**를 추출하여 쉼표(,)로 구분하여 나열해라. 오직 키워드 목록만 출력하라.

[핵심 키워드 목록]
""",
    template_format="jinja2",
)

# 11. 추천 인삿말 생성 프롬프트
recommendation_intro_prompt = PromptTemplate(
    input_variables=["query", "persona_info"],
    template="""
사용자의 도서 추천 요청은 다음과 같습니다:
"{{ query }}"

챗봇은 다음과 같은 역할을 맡고 있습니다:
"{{ persona_info }}"

위 내용을 고려하여, 추천 리스트를 자연스럽게 시작하는 인삿말을 한두 문장으로 만들어주세요.
예: "이런 책들은 어떠세요?", "다음 책들을 추천드려요!" 등
""",
    template_format="jinja2",
)

## 12. 추천 마무리 멘트 프롬프트
recommendation_outro_prompt = PromptTemplate(
    input_variables=["query", "persona_info"],
    template="""
[사용자의 추천 요청]
"{{ query }}"

[챗봇의 역할]
{{ persona_info }}

위 정보를 고려하여, 추천 리스트의 **마무리 인사말**을 자연스럽고 친근한 말투로 1~2문장 생성하세요.  
사용자가 마음에 드는 책을 발견했기를 바라는 따뜻한 톤이면 좋습니다. 너무 딱딱하지 않게, 대화를 부드럽게 끝맺는 느낌으로 작성하세요.

[예시]
- "하나쯤 읽고 싶은 책이 생기셨길 바라요 :)"
- "마음에 드는 책이 있다면 참 좋겠네요!"
- "더 궁금한 책이 있다면 언제든지 말씀해주세요."

[추천 마무리 멘트]
""",
    template_format="jinja2",
)

In [67]:
async def async_invoke(chain: LLMChain, vars_dict: dict, step_name: str) -> dict:
    """LLMChain을 비동기로 실행하고 로깅"""
    try:
        # 입력 변수 일부 로깅
        log_vars = {
            k: (v[:100] + "..." if isinstance(v, str) and len(v) > 100 else v)
            for k, v in vars_dict.items()
        }
        logger.debug(f"[{step_name}] Chain 호출 시작. 입력 변수 일부: {log_vars}")
        # chain.invoke를 별도 스레드에서 실행하여 비동기 처리
        result = await asyncio.to_thread(chain.invoke, vars_dict)
        result_text = result.get("text", "")
        # 결과 텍스트 일부 로깅
        log_result = (
            result_text[:200] + "..." if len(result_text) > 200 else result_text
        )
        logger.debug(f"[{step_name}] Chain 호출 완료. 결과 일부: {log_result}")
        return result
    except Exception as e:
        logger.error(f"[{step_name}] Chain 호출 중 예외 발생: {e}", exc_info=True)
        return {"text": ""}  # 오류 시 빈 결과 반환


async def async_invoke_llm(prompt: str, step_name: str) -> str:
    """LLM 직접 호출을 비동기로 실행하고 로깅"""
    try:
        # 프롬프트 일부 로깅
        log_prompt = prompt[:200] + "..." if len(prompt) > 200 else prompt
        logger.debug(f"[{step_name}] LLM 호출 시작. 프롬프트 일부: {log_prompt}")
        # llm.invoke를 별도 스레드에서 실행
        response = await asyncio.to_thread(llm_clova.invoke, prompt)

        # 응답 객체에서 텍스트 추출
        if hasattr(response, "content"):
            result_text = response.content.strip()
        elif isinstance(response, str):  # ChatClovaX가 문자열 직접 반환하는 경우
            result_text = response.strip()
        else:
            result_text = str(response).strip()

        # 응답 텍스트 일부 로깅
        log_result = (
            result_text[:200] + "..." if len(result_text) > 200 else result_text
        )
        logger.debug(f"[{step_name}] LLM 호출 완료. 응답 일부: {log_result}")
        return result_text
    except Exception as e:
        logger.error(f"[{step_name}] LLM 호출 중 예외 발생: {e}", exc_info=True)
        return ""  # 오류 시 빈 문자열

In [68]:
def _extract_metadata_field(doc: Document, field: str, default: str = "N/A") -> str:
    """
    Document의 다양한 metadata 구조에서 필드 값을 추출 (예: title, ISBN 등).
    """
    candidates = []

    # Case 1: 평평한 구조
    candidates.append(doc.metadata)

    # Case 2: 1단계 중첩
    if "metadata" in doc.metadata and isinstance(doc.metadata["metadata"], dict):
        candidates.append(doc.metadata["metadata"])

        # Case 3: 2단계 중첩
        inner = doc.metadata["metadata"]
        if "metadata" in inner and isinstance(inner["metadata"], dict):
            candidates.append(inner["metadata"])

    # Case 4: Elasticsearch 구조 (_source.metadata)
    if "_source" in doc.metadata and isinstance(doc.metadata["_source"], dict):
        source_meta = doc.metadata["_source"].get("metadata", {})
        if isinstance(source_meta, dict):
            candidates.append(source_meta)

    # 실제 값 추출
    for meta in candidates:
        if field in meta:
            return (
                meta[field].strip()
                if isinstance(meta[field], str)
                else str(meta[field])
            )

    return default

In [ ]:
class BaseRAGPipeline:
    def __init__(
        self, config, llm, embeddings, vectorstore, es_store, retriever, documents
    ):
        self.config = config  # config에 페르소나별 템플릿 및 가중치 포함
        self.llm = llm
        self.embeddings = embeddings
        self.vectorstore = vectorstore
        self.es_store = es_store
        self.retriever = retriever
        self.documents = documents

        self.user_history: List[str] = []
        self.llm_history: List[str] = []
        self.user_preferences: Dict[str, List[str]] = self._initialize_preferences()
        self.preferences_text: str = "수집된 선호도 없음"
        self.preference_update_count: int = 0
        self.last_recommendations: List[Document] = []
        self.last_action: Optional[str] = None

        # 페르소나별 프롬프트 체인 생성
        self.extract_pref_chain = LLMChain(
            llm=self.llm, prompt=self.config["pref_extraction_template"]
        )
        self.decision_chain = LLMChain(
            llm=self.llm, prompt=self.config.get("decision_template", None)
        )
        self.final_query_generation_chain = LLMChain(
            llm=self.llm, prompt=self.config["final_query_template"]
        )
        self.refine_chain = LLMChain(
            llm=self.llm, prompt=self.config["refine_template"]
        )
        self.query_expansion_chain = LLMChain(
            llm=self.llm, prompt=self.config["expansion_template"]
        )
        self.re_ranking_chain = LLMChain(
            llm=self.llm, prompt=self.config.get("re_ranking_template", None)
        )
        self.hyde_generation_chain = LLMChain(
            llm=self.llm, prompt=self.config.get("hyde_generation_template", None)
        )
        self.hyde_keyword_chain = LLMChain(
            llm=self.llm, prompt=self.config.get("hyde_keyword_template", None)
        )
        self.recommendation_intro_chain = LLMChain(
            llm=self.llm, prompt=self.config["recommendation_intro_template"]
        )
        self.recommendation_outro_chain = LLMChain(
            llm=self.llm, prompt=self.config["recommendation_outro_template"]
        )

    def _initialize_preferences(self) -> Dict[str, List[str]]:
        return {
            "title": [],
            "author": [],
            "category": [],
            "author_intro": [],
            "book_intro": [],
            "table_of_contents": [],
            "purpose": [],
            "implicit info": [],
        }

    def robust_parse_decision_response(
        self, response_text: str
    ) -> tuple[Optional[str], str]:
        action = None
        additional_question = ""
        action_match = re.search(r'행동\s*[:：]\s*"?([^"\n]+)"?', response_text)
        if action_match:
            action = action_match.group(1).strip().lower()
            if action not in ["추천", "추가 질문"]:
                logger.warning(
                    f"알 수 없는 행동 값 파싱됨: '{action}'. '추가 질문'으로 처리."
                )
                action = "추가 질문"
        follow_match = re.search(
            r'추가\s*질문\s*[:：]\s*"?(.+)"?', response_text, re.DOTALL
        )
        if follow_match:
            additional_question = follow_match.group(1).strip()
        if action == "추가 질문" and not additional_question:
            additional_question = "어떤 점이 궁금하신가요? 또는 어떤 책을 찾으시나요?"
            logger.warning(
                f"행동은 '추가 질문'이나 질문 내용 없음. 기본 질문 사용: '{additional_question}'"
            )
        if not action:
            logger.warning(
                f"행동 결정 파싱 실패: '{response_text}'. 기본 '추가 질문'으로 처리."
            )
            action = "추가 질문"
            if not additional_question:
                additional_question = "요청을 이해하기 어려웠습니다. 어떤 책을 찾으시는지 다시 말씀해주시겠어요?"
        logger.info(
            f"행동 결정 파싱 결과: 행동='{action}', 추가 질문='{additional_question[:50]}...'"
        )
        return action, additional_question

    async def update_preferences_from_input(self, user_input: str) -> None:
        logger.info(f"사용자 입력에서 선호도 추출 시작: '{user_input[:100]}...'")
        extract_result = await async_invoke(
            self.extract_pref_chain, {"text": user_input}, "선호도 추출"
        )
        extracted_text = extract_result.get("text", "{}")
        extracted_prefs: Dict[str, List[str]] = {}
        try:
            json_match = re.search(r"\{.*\}", extracted_text, re.DOTALL)
            if json_match:
                extracted_prefs_raw = json.loads(json_match.group(0))
                defined_keys = self.user_preferences.keys()
                for key, value in extracted_prefs_raw.items():
                    if key in defined_keys:
                        vals_to_add = []
                        if isinstance(value, list):
                            vals_to_add = [
                                str(item).strip()
                                for item in value
                                if item and str(item).strip()
                            ]
                        elif isinstance(value, str) and value.strip():
                            vals_to_add = [value.strip()]
                        if vals_to_add:
                            extracted_prefs[key] = vals_to_add
                    else:
                        logger.warning(
                            f"추출된 선호도 키 '{key}'가 정의된 형식에 없음. 무시."
                        )
            else:
                logger.warning(
                    f"선호도 추출 결과에서 JSON 객체를 찾을 수 없음: {extracted_text}"
                )
        except json.JSONDecodeError as e:
            logger.error(
                f"선호도 추출 결과 JSON 파싱 실패: {e}. 원본 텍스트: {extracted_text}"
            )
        except Exception as e:
            logger.error(f"선호도 추출/처리 중 예외 발생: {e}", exc_info=True)
        if not extracted_prefs:
            logger.info("새로 추출된 유효한 선호도 정보가 없음")
            return
        updated_something = False
        for key, new_values in extracted_prefs.items():
            existing_values_set = set(self.user_preferences.get(key, []))
            added_values = [v for v in new_values if v not in existing_values_set]
            if added_values:
                self.user_preferences[key].extend(added_values)
                updated_something = True
                logger.info(f"선호도 업데이트됨 [{key}]: {self.user_preferences[key]}")
        if updated_something:
            self.preference_update_count += 1
            logger.info(
                f"선호도 업데이트 완료. 누적 업데이트 횟수: {self.preference_update_count}"
            )
            self._update_preferences_text()
        else:
            logger.info("기존 선호도에서 변경된 내용 없음.")

    def _update_preferences_text(self):
        pref_items = []
        display_key_map = {
            "title": "관련 제목",
            "author": "선호 저자",
            "category": "선호 장르/분류",
            "author_intro": "저자 관련 요구",
            "book_intro": "내용 관련 요구",
            "table_of_contents": "목차/키워드 요구",
            "purpose": "독서 목적",
            "implicit info": "기타 희망 사항/분위기",
        }
        for key, values in self.user_preferences.items():
            if values:
                display_key = display_key_map.get(key, key)
                pref_items.append(f"- {display_key}: {', '.join(values)}")
        self.preferences_text = (
            "\n".join(pref_items) if pref_items else "수집된 선호도 없음"
        )
        logger.debug(f"업데이트된 선호도 요약 텍스트:\n{self.preferences_text}")

    async def get_final_query(self, current_user_query: str) -> str:
        logger.info("최종 검색 쿼리 생성 시작")
        persona_info = self.config.get("persona_info", "기본 정보")
        final_query_vars = {
            "history": "\n".join(self.user_history[-5:] + self.llm_history[-5:]),
            "query": current_user_query,
            "persona_info": persona_info,
            "preferences": self.preferences_text,
        }
        result_gen = await async_invoke(
            self.final_query_generation_chain, final_query_vars, "선호도 종합 쿼리 생성"
        )
        generated_query = result_gen.get("text", "").strip()
        logger.info(f"LLM 생성 쿼리 (정제 전): '{generated_query}'")
        query_to_use = generated_query
        if generated_query:
            refine_result = await async_invoke(
                self.refine_chain, {"query": generated_query}, "쿼리 정제"
            )
            refined_query = refine_result.get("text", "").strip().strip('"')
            logger.info(f"정제된 쿼리: '{refined_query}'")
            negative_keywords = [
                "없",
                "못",
                "않",
                "오류",
                "잘못",
                "알 수 없",
                "죄송",
                "필요",
            ]
            is_invalid_refinement = (
                not refined_query
                or len(refined_query) < 3
                or any(keyword in refined_query for keyword in negative_keywords)
                or "{" in refined_query
                or "}" in refined_query
                or refined_query.lower() == generated_query.lower()
            )
            if is_invalid_refinement:
                logger.warning(
                    f"정제된 쿼리('{refined_query}')가 유효하지 않아 정제 전 쿼리('{generated_query}') 사용."
                )
            else:
                query_to_use = refined_query
        else:
            logger.warning("선호도 종합 쿼리 생성 실패. 원본 사용자 쿼리 사용.")
            query_to_use = current_user_query
        if not query_to_use or len(query_to_use) < 3:
            logger.warning(
                f"최종 결정된 쿼리('{query_to_use}')가 너무 짧거나 비어있어 원본 사용자 쿼리('{current_user_query}') 사용."
            )
            query_to_use = current_user_query
        logger.info(f"최종 결정된 검색 쿼리: '{query_to_use}'")
        return query_to_use

    async def _summarize_chunk_with_llm(self, text: str) -> str:
        if not text or len(text.strip()) < MIN_INFO_LENGTH:
            return "요약할 정보가 충분하지 않습니다."
        max_len = 4000
        truncated_text = text[:max_len].strip()
        if not truncated_text:
            return "요약할 정보가 없습니다."
        prompt = f"""
        다음 책 정보를 2~3문장으로 핵심 내용만 요약해줘.  
        말투는 너무 딱딱하지 않고, 사용자에게 설명하듯 **자연스럽고 친근한 어조**로 해줘.  
        예를 들어 "~있다" 대신 "~있어요", "~이다" 대신 "~예요"처럼 말해줘.:\n\n{truncated_text}\n\n요약:"
        """
        summary = await async_invoke_llm(prompt, "청크 요약")
        if (
            not summary
            or len(summary) < 10
            or "요약할 정보가" in summary
            or "죄송" in summary
            or "모르겠" in summary
        ):
            fallback_summary = text[:300].strip() + ("..." if len(text) > 300 else "")
            logger.warning(
                f"LLM 요약 실패 또는 부적절. Fallback 요약 사용: '{fallback_summary[:100]}...'"
            )
            return (
                fallback_summary
                if fallback_summary
                else "별도의 상세 정보가 충분치 않습니다."
            )
        return summary

    def _merge_documents_by_isbn(self, isbn: str) -> Optional[Document]:
        if not isbn:
            logger.warning("ISBN 없이 문서 병합 시도됨.")
            return None
        docs_for_isbn = [
            doc
            for doc in self.documents
            if str(doc.metadata.get("ISBN", "")).strip() == str(isbn).strip()
        ]
        if not docs_for_isbn:
            logger.warning(
                f"ISBN '{isbn}'에 해당하는 문서를 마스터 목록에서 찾을 수 없음."
            )
            return None
        combined_text = "\n\n---\n\n".join(
            doc.page_content for doc in docs_for_isbn if doc.page_content
        ).strip()
        merged_meta = dict(docs_for_isbn[0].metadata)
        logger.debug(
            f"ISBN '{isbn}' 문서 병합 완료 (전체 원본 기준). 병합된 청크 수: {len(docs_for_isbn)}, 총 텍스트 길이: {len(combined_text)}"
        )
        return Document(page_content=combined_text, metadata=merged_meta)

    async def _embedding_rerank_documents(
        self, query: str, documents: List[Document]
    ) -> List[Document]:
        if not documents:
            logger.info("리랭킹할 문서가 없습니다.")
            return []
        embedding_tasks = {
            "main": asyncio.to_thread(self.embeddings.embed_query, query)
        }
        if self.user_preferences.get("author"):
            embedding_tasks["author"] = asyncio.to_thread(
                self.embeddings.embed_query, self.user_preferences["author"][0]
            )
        if self.user_preferences.get("title"):
            embedding_tasks["title"] = asyncio.to_thread(
                self.embeddings.embed_query, self.user_preferences["title"][0]
            )
        if self.user_preferences.get("category"):
            embedding_tasks["category"] = asyncio.to_thread(
                self.embeddings.embed_query, self.user_preferences["category"][0]
            )
        query_embeddings = await asyncio.gather(*embedding_tasks.values())
        query_embedding_map = dict(zip(embedding_tasks.keys(), query_embeddings))
        # 페르소나별 retrieval_weights 사용 (없으면 기본값)
        weights = self.config.get(
            "retrieval_weights",
            {"main": 0.1, "author": 0.3, "title": 0.1, "category": 0.5},
        )
        scored_docs = []
        for doc in documents:
            score_map: Dict[str, float] = {}
            real_meta = doc.metadata.get("metadata", doc.metadata)
            doc_embedding = await asyncio.to_thread(
                self.embeddings.embed_query, doc.page_content
            )
            score_map["main"] = cosine_similarity(
                [query_embedding_map["main"]], [doc_embedding]
            )[0][0]
            author_text = real_meta.get("author", "")
            if author_text and "author" in query_embedding_map:
                author_emb = await asyncio.to_thread(
                    self.embeddings.embed_query, author_text
                )
                score_map["author"] = cosine_similarity(
                    [query_embedding_map["author"]], [author_emb]
                )[0][0]
            title_text = real_meta.get("title", "")
            if title_text and "title" in query_embedding_map:
                title_emb = await asyncio.to_thread(
                    self.embeddings.embed_query, title_text
                )
                score_map["title"] = cosine_similarity(
                    [query_embedding_map["title"]], [title_emb]
                )[0][0]
            category_text = real_meta.get("category", "")
            if category_text and "category" in query_embedding_map:
                category_emb = await asyncio.to_thread(
                    self.embeddings.embed_query, category_text
                )
                score_map["category"] = cosine_similarity(
                    [query_embedding_map["category"]], [category_emb]
                )[0][0]
            valid_keys = score_map.keys()
            total_weight = sum(weights[k] for k in valid_keys)
            final_score = sum(
                (weights[k] / total_weight) * score_map[k] for k in valid_keys
            )
            scored_docs.append((doc, final_score))
        sorted_docs = sorted(scored_docs, key=lambda x: x[1], reverse=True)
        logger.info("리랭킹 완료 (정규화 가중치 방식)")
        for i, (doc, score) in enumerate(sorted_docs):
            title = _extract_metadata_field(doc, "title", default="제목 없음")
            isbn = _extract_metadata_field(doc, "ISBN", default="N/A")
            logger.info(
                f"{i+1}. 제목: {title} | ISBN: {isbn} | 점수: {round(score, 4)}"
            )

        reranked_docs = [doc for doc, score in sorted_docs if score >= 0.5]
        logger.info("리랭킹 완료 (0.5 이상 필터 적용됨)")
        return reranked_docs

    async def _expand_query(self, query: str) -> List[str]:
        logger.info(f"검색 쿼리 확장 시작: '{query}'")
        expansion_result = await async_invoke(
            self.query_expansion_chain, {"query": query}, "검색 쿼리 확장"
        )
        expanded_queries_text = expansion_result.get("text", "").strip()
        expanded_queries = []
        for line in expanded_queries_text.splitlines():
            line_stripped = line.strip()
            if not line_stripped:
                continue
            query_part = re.sub(r"^\d+\.\s*", "", line_stripped).strip()
            if query_part and query_part.lower() != query.lower():
                expanded_queries.append(query_part)
        final_expanded_queries = expanded_queries[:3]
        logger.info(
            f"생성된 확장 쿼리 ({len(final_expanded_queries)}개): {final_expanded_queries}"
        )
        return final_expanded_queries

    async def _retrieve_documents(
        self, query: str, use_hyde: bool = False
    ) -> List[Document]:
        retrieval_query = query
        hyde_summary = ""
        if use_hyde and self.user_preferences.get("implicit info"):
            logger.info("HyDE 활성화: 가상 문서 요약 및 키워드 추출 시도")
            hyde_result = await async_invoke(
                self.hyde_generation_chain, {"query": query}, "HyDE 가상 문서 생성"
            )
            hyde_summary = hyde_result.get("text", "").strip()
            if hyde_summary:
                logger.info(f"생성된 가상 문서 요약: '{hyde_summary[:200]}...'")
                keyword_result = await async_invoke(
                    self.hyde_keyword_chain,
                    {"hyde_summary": hyde_summary},
                    "HyDE 키워드 추출",
                )
                hyde_keywords_text = keyword_result.get("text", "").strip()
                hyde_keywords = [
                    k.strip() for k in hyde_keywords_text.split(",") if k.strip()
                ]
                if hyde_keywords:
                    logger.info(f"추출된 HyDE 키워드: {hyde_keywords}")
                    retrieval_query = f"{query} {' '.join(hyde_keywords)}"
                    logger.info(f"HyDE 적용된 검색 쿼리: '{retrieval_query}'")
                else:
                    logger.warning(
                        "HyDE 요약에서 키워드를 추출하지 못했습니다. 원본 쿼리 사용."
                    )
            else:
                logger.warning("HyDE 가상 문서 요약 생성 실패. 원본 쿼리 사용.")
        elif use_hyde:
            logger.info("HyDE 조건 미충족 ('implicit info' 없음). 일반 쿼리 검색 수행.")
        else:
            logger.info("일반 쿼리 검색 수행.")
        logger.info(f"최종 리트리버 호출 시작 (쿼리: '{retrieval_query[:100]}...')")
        try:
            retrieved_docs = await self.retriever.aget_relevant_documents(
                retrieval_query
            )
            logger.info(f"검색된 문서 수 (ISBN 병합 완료됨): {len(retrieved_docs)}")
        except Exception as e:
            logger.error(
                f"문서 검색 실패 (쿼리: '{retrieval_query}'): {e}", exc_info=True
            )
            retrieved_docs = []
        return retrieved_docs

    async def _generate_recommendations(self, final_query: str) -> str:
        logger.info(f"추천 생성 시작. 최종 쿼리: '{final_query}'")
        use_hyde = False
        use_expansion = False
        has_author = bool(self.user_preferences.get("author"))
        has_title = bool(self.user_preferences.get("title"))
        has_implicit = bool(self.user_preferences.get("implicit info"))
        if not has_author and not has_title and has_implicit:
            use_hyde = True
            use_expansion = True
        initial_docs = await self._retrieve_documents(final_query, use_hyde=use_hyde)
        all_docs_to_consider = list(initial_docs)
        processed_queries = {final_query.lower()}
        if use_expansion:
            expanded_queries = await self._expand_query(final_query)
            expansion_tasks = []
            if expanded_queries:
                logger.info(
                    f"확장 쿼리 ({len(expanded_queries)}개)로 추가 검색 수행..."
                )
                for eq in expanded_queries:
                    eq_lower = eq.lower()
                    if eq_lower not in processed_queries:
                        expansion_tasks.append(
                            self._retrieve_documents(eq, use_hyde=False)
                        )
                        processed_queries.add(eq_lower)
                if expansion_tasks:
                    expansion_results = await asyncio.gather(*expansion_tasks)
                    existing_isbns = {
                        doc.metadata.get("ISBN")
                        for doc in all_docs_to_consider
                        if doc.metadata.get("ISBN")
                    }
                    for docs_list in expansion_results:
                        for doc in docs_list:
                            isbn = doc.metadata.get("ISBN")
                            if isbn and isbn not in existing_isbns:
                                all_docs_to_consider.append(doc)
                                existing_isbns.add(isbn)
        logger.info(
            f"초기 및 확장 검색 후 고려할 총 고유 문서 수: {len(all_docs_to_consider)}"
        )

        for i, doc in enumerate(all_docs_to_consider):
            metadata_content = doc.metadata if doc.metadata else {}
            logger.debug(
                f"문서 {i+1} Metadata: {json.dumps(metadata_content, indent=2, ensure_ascii=False)}"
            )
            try:
                isbn_check = metadata_content.get("ISBN", "ISBN 키 없음")
                title_check = metadata_content.get("title", "Title 키 없음")
                logger.debug(f"  -> 확인: ISBN='{isbn_check}', Title='{title_check}'")
            except Exception as e:
                logger.error(f"  -> 메타데이터 접근 오류 발생: {e}")
        logger.debug("--- 리랭킹 입력 데이터 확인 완료 ---")
        if not all_docs_to_consider:
            logger.warning("검색 및 확장 결과 문서를 찾지 못했습니다. 추천 생성 불가.")
            return "죄송합니다, 해당 조건에 맞는 책을 찾지 못했습니다. 다른 조건으로 다시 시도해 보시겠어요?"
        logger.info("리랭킹 수행...")
        ranked_docs = await self._embedding_rerank_documents(
            final_query, all_docs_to_consider
        )
        if not ranked_docs:
            logger.warning("리랭킹 결과 유효한 문서가 없습니다. 추천 생성 불가.")
            self.last_recommendations = []  # 후속 질문 방지
            return "죄송합니다, 요청하신 조건에 맞는 책을 찾지 못했습니다. 다른 조건으로 다시 시도해 보시겠어요?"
        top_docs = ranked_docs[:3]
        logger.info(f"최종 추천 후보 문서 수: {len(top_docs)}")
        recommendations = []
        self.last_recommendations = []
        processed_isbns = set()
        for rank, doc in enumerate(top_docs):
            isbn = _extract_metadata_field(doc, "ISBN")
            title = _extract_metadata_field(doc, "title", default="제목 없음")
            logger.debug(
                f"추천 후보 처리 중 (Rank {rank+1}): ISBN={isbn}, Title={title}"
            )
            if not isbn or isbn == "N/A":
                logger.warning(
                    f"Rank {rank+1} 추천 후보에 ISBN 없음. 건너뜀. Title: {title}"
                )
                continue
            if isbn in processed_isbns:
                logger.warning(f"중복 ISBN '{isbn}' 추천 목록에 이미 존재. 건너뜀.")
                continue
            full_merged_doc = self._merge_documents_by_isbn(isbn)
            if not full_merged_doc:
                logger.error(
                    f"치명적 오류: 리랭킹된 문서(ISBN: {isbn})의 전체 정보를 병합하지 못했습니다. 추천 목록에서 제외."
                )
                continue
            self.last_recommendations.append(full_merged_doc)
            processed_isbns.add(isbn)
            metadata = full_merged_doc.metadata
            title = metadata.get("title", "제목 정보 없음")
            author = metadata.get("author", "저자 정보 없음")
            book_cover = metadata.get("book_cover", "표지 정보 없음")
            book_intro = extract_field(full_merged_doc.page_content, "책소개")
            publisher_review = extract_field(full_merged_doc.page_content, "출판사리뷰")
            recommendation_field = extract_field(full_merged_doc.page_content, "추천사")
            text_for_summary = ""
            if book_intro and len(book_intro.strip()) >= MIN_INFO_LENGTH:
                text_for_summary = book_intro
            elif publisher_review and len(publisher_review.strip()) >= MIN_INFO_LENGTH:
                text_for_summary = publisher_review
            elif (
                recommendation_field
                and len(recommendation_field.strip()) >= MIN_INFO_LENGTH
            ):
                text_for_summary = recommendation_field
            else:
                page_content_cleaned = re.sub(
                    r"^\s*.*?\s*[:：]\s*",
                    "",
                    full_merged_doc.page_content,
                    flags=re.MULTILINE,
                ).strip()
                text_for_summary = page_content_cleaned[:500]
            if text_for_summary and len(text_for_summary.strip()) >= MIN_INFO_LENGTH:
                summary = await self._summarize_chunk_with_llm(text_for_summary)
            else:
                summary = "책에 대한 상세 설명이 부족합니다."
            recommendation_text = f"{rank+1}. \r\n표지: {book_cover}\r\n제목: {title}\r\n저자: {author}\r\n- 추천 이유: {summary}"
            recommendations.append(recommendation_text)
            logger.debug(f"추천 문구 생성됨: {title}")
        if self.last_recommendations:
            rec_titles = [
                d.metadata.get("title", "N/A") for d in self.last_recommendations
            ]
            logger.info(
                f"'_generate_recommendations' 종료. last_recommendations 업데이트 ({len(self.last_recommendations)}개): {rec_titles}"
            )
        else:
            logger.warning(
                "'_generate_recommendations' 종료. 최종 추천 목록(last_recommendations)이 비어있음!"
            )
        if not recommendations:
            return "추천할 만한 책을 찾지 못했습니다. 조건을 바꿔서 다시 질문해 주시겠어요?"
        persona_info_map = {
            "Literature": "감성, 현재 기분, 선호하는 문학 장르 및 작가 스타일",
            "Science": "독자의 지식 수준(초심자/전문가), 관심 기술 분야, 문제 해결 목표",
            "General": "선호 장르, 책을 찾는 이유, 원하는 분위기나 난이도",
        }
        intro_vars = {
            "query": final_query,
            "persona_info": persona_info_map.get(self.config.get("persona", "General")),
        }
        intro_result = await async_invoke(
            self.recommendation_intro_chain, intro_vars, "추천 인삿말 생성"
        )
        recommendation_intro = intro_result.get("text", "").strip()
        if not recommendation_intro:
            recommendation_intro = "이런 책들은 어떠세요?"

        # 추천 아웃트로 생성
        outro_vars = {
            "query": final_query,
            "persona_info": persona_info_map.get(self.config.get("persona", "General")),
        }
        try:
            outro_result = await async_invoke(
                self.recommendation_outro_chain, outro_vars, "추천 마무리 멘트 생성"
            )
            recommendation_outro = outro_result.get("text", "").strip()
            if not recommendation_outro or len(recommendation_outro) < 3:
                recommendation_outro = "마음에 드는 책이 있으셨으면 좋겠어요 :)"
        except Exception as e:
            logger.warning(f"추천 아웃트로 생성 실패: {e}")
            recommendation_outro = "마음에 드는 책이 있으셨으면 좋겠어요 :)"

        # 최종 답변 조립
        final_answer = (
            recommendation_intro
            + "\n\n"
            + "\n\n".join(recommendations)
            + "\n\n"
            + recommendation_outro
        )
        logger.info("추천 응답 생성 완료.")
        return final_answer

    async def handle_followup_query(self, followup_query: str) -> tuple[bool, str]:
        logger.info(f"후속 질문 처리 시작. Query: '{followup_query}'")
        if not self.last_recommendations:
            logger.error(
                "handle_followup_query 진입 오류: last_recommendations가 비어 있음!"
            )
            return False, ""
        rec_info = []
        for i, doc in enumerate(self.last_recommendations):
            title = doc.metadata.get("title", "제목 없음")
            isbn = doc.metadata.get("ISBN", "NO_ISBN")
            snippet = await self._summarize_chunk_with_llm(doc.page_content[:500])
            rec_info.append(f"{i+1}. 제목: {title}, ISBN: {isbn}\n   요약: {snippet}")
        rec_info_str = "\n".join(rec_info)
        prompt = f"""
        이전에 다음 책들을 추천했습니다:
        {rec_info_str}

        사용자의 후속 질문은 다음과 같습니다: "{followup_query}"

        이 질문이 위 추천 목록과 관련된 후속 질문인지, 아니면 완전히 새로운 질문인지 판단하고, 후속 질문이라면 그 의도를 분석하여 다음 JSON 형식 중 **하나만** 출력해라. 새로운 질문이면 {{"action": "새 질문", "ISBN": null, "query": null}} 형식으로 출력하라. **절대로 다른 설명이나 대화 없이 오직 JSON 객체 하나만 출력해야 한다.**
        [후속 질문 의도 분류 및 JSON 형식]
        - 특정 책 상세 정보 요청: {{"action": "상세", "ISBN": "<요청된 책의 ISBN>", "query": "{followup_query}"}}
        - 특정 책과 유사한 책 추천 요청: {{"action": "유사", "ISBN": "<기준 책의 ISBN>", "query": "<유사성 관련 사용자 언급>"}}
        - 추천된 책들 비교 요청: {{"action": "비교", "ISBN": "<비교 대상 ISBN 목록 (쉼표 구분)>", "query": "{followup_query}"}}
        - 추천 결과에 대한 피드백/불만: {{"action": "피드백", "ISBN": null, "query": "{followup_query}"}}
        - 완전히 새로운 질문: {{"action": "새 질문", "ISBN": null, "query": null}}
        [분석 결과 (JSON 객체만 출력)]
        """
        try:
            result_text = await async_invoke_llm(prompt, "후속 질문 의도 분석")
            logger.debug(f"후속 질문 의도 분석 LLM 원본 응답: {result_text}")
            analysis_result = None
            json_match = re.search(r"\{.*\}", result_text, re.DOTALL)
            if json_match:
                try:
                    analysis_result = json.loads(json_match.group(0))
                except json.JSONDecodeError:
                    logger.error(
                        f"후속 질문 의도 분석 결과 JSON 파싱 실패 (JSON 형식 오류): {result_text}"
                    )
                    analysis_result = {"action": "새 질문"}
            else:
                if (
                    "새 질문" in result_text
                    or '"action": "새 질문"' in result_text.replace(" ", "")
                ):
                    logger.warning(
                        f"후속 질문 의도 분석 JSON 객체 미발견, '새 질문' 패턴 감지: {result_text}"
                    )
                    analysis_result = {"action": "새 질문"}
                else:
                    logger.error(
                        f"후속 질문 의도 분석 결과 JSON 파싱 실패 및 '새 질문' 패턴 미감지: {result_text}"
                    )
                    return False, ""
            action = analysis_result.get("action", "새 질문")
            isbn_str = analysis_result.get("ISBN", "")
            query_part = analysis_result.get("query", followup_query)
            logger.info(
                f"후속 질문 분석 결과: action='{action}', ISBN='{isbn_str}', query='{query_part[:50]}...'"
            )
            if action == "새 질문":
                return False, ""
            isbn_list = (
                [s.strip() for s in str(isbn_str).split(",") if s.strip()]
                if isbn_str
                else []
            )
            target_isbn = isbn_list[0] if isbn_list else None
            if action == "상세":
                if not target_isbn:
                    return (
                        True,
                        "어떤 책에 대해 더 알고 싶으신지 알려주시겠어요? (예: 첫 번째 책 또는 책 제목)",
                    )
                target_doc = next(
                    (
                        doc
                        for doc in self.last_recommendations
                        if str(doc.metadata.get("ISBN", "")).strip()
                        == str(target_isbn).strip()
                    ),
                    None,
                )
                if target_doc:
                    logger.debug(
                        f"상세 정보 요청: ISBN '{target_isbn}' 문서 찾음: Title='{target_doc.metadata.get('title')}'"
                    )
                    detail_prompt = f"""
                    다음은 사용자가 문의한 '{target_doc.metadata.get("title", "해당 책")}'에 대한 정보입니다. 이 정보를 바탕으로 사용자 질문 "{query_part}"에 답하거나, 특별한 질문이 없다면 책에 대해 자연스럽게 더 자세히 설명해주세요.
                    [책 정보 요약]
                    제목: {target_doc.metadata.get("title", "정보 없음")}
                    저자: {target_doc.metadata.get("author", "정보 없음")}
                    분류: {target_doc.metadata.get("category", "정보 없음")}
                    페이지: {target_doc.metadata.get("page", "정보 없음")} 쪽
                    가격: {target_doc.metadata.get("price", "정보 없음")} 원
                    [책 소개 및 내용 (일부)]
                    {target_doc.page_content[:3000]}...
                    [답변 또는 상세 설명]
                    """
                    detailed_info = await async_invoke_llm(
                        detail_prompt, "후속 상세 설명 생성"
                    )
                    return True, (
                        detailed_info
                        if detailed_info
                        else "죄송합니다, 요청하신 내용에 대한 추가 정보를 제공하기 어렵습니다."
                    )
                else:
                    available_isbns = [
                        d.metadata.get("ISBN") for d in self.last_recommendations
                    ]
                    return (
                        True,
                        f"죄송합니다, 요청하신 ISBN({target_isbn})의 책 정보를 찾을 수 없습니다. 현재 추천된 책들의 ISBN은 다음과 같습니다: {available_isbns}",
                    )
            elif action == "유사":
                if not target_isbn:
                    return (
                        True,
                        "어떤 책과 유사한 책을 찾으시는지 알려주시겠어요? (예: 두 번째 책 같은 스타일)",
                    )
                base_doc = next(
                    (
                        doc
                        for doc in self.last_recommendations
                        if str(doc.metadata.get("ISBN", "")).strip()
                        == str(target_isbn).strip()
                    ),
                    None,
                )
                if base_doc:
                    logger.debug(
                        f"유사 책 요청: 기준 ISBN '{target_isbn}' 문서 찾음: Title='{base_doc.metadata.get('title')}'"
                    )
                    base_title = base_doc.metadata.get("title", "")
                    base_author = base_doc.metadata.get("author", "")
                    base_category = base_doc.metadata.get("category", "")
                    similarity_aspects = (
                        f"{base_category} 장르"
                        if base_category
                        else f"'{base_title}'와 비슷한"
                    )
                    if base_author:
                        similarity_aspects += f" {base_author} 작가 스타일"
                    user_refinement = (
                        f" 그리고 '{query_part}' 특징을 가진"
                        if query_part and query_part != followup_query
                        else ""
                    )
                    new_query = f"{similarity_aspects}{user_refinement} 책 추천"
                    logger.info(f"유사 책 추천을 위한 새 쿼리 생성: {new_query}")
                    recommendation_result = await self._generate_recommendations(
                        new_query
                    )
                    return (
                        True,
                        f"네, '{base_title}'와(과) 비슷한 다른 책을 찾아볼게요.\n\n"
                        + recommendation_result,
                    )
                else:
                    available_isbns = [
                        d.metadata.get("ISBN") for d in self.last_recommendations
                    ]
                    return (
                        True,
                        f"죄송합니다, 기준이 되는 ISBN({target_isbn})의 책 정보를 찾을 수 없습니다. 현재 추천된 책들의 ISBN은 다음과 같습니다: {available_isbns}",
                    )
            elif action == "비교":
                if len(isbn_list) < 2:
                    return (
                        True,
                        "비교할 책을 두 권 이상 알려주시겠어요? (예: 첫 번째랑 세 번째 책 비교해주세요)",
                    )
                valid_comparison_isbns = [
                    isbn
                    for isbn in isbn_list
                    if any(
                        str(d.metadata.get("ISBN", "")).strip() == str(isbn).strip()
                        for d in self.last_recommendations
                    )
                ]
                if len(valid_comparison_isbns) < 2:
                    available_isbns = [
                        d.metadata.get("ISBN") for d in self.last_recommendations
                    ]
                    return (
                        True,
                        f"죄송합니다, 비교 요청하신 책({isbn_list}) 중 일부를 찾을 수 없거나 유효하지 않습니다. 현재 추천된 책 ISBN: {available_isbns}",
                    )
                comparison_result = await self._handle_comparison(
                    query_part, valid_comparison_isbns
                )
                return True, comparison_result
            elif action == "피드백":
                logger.info(f"사용자 피드백/불만 처리 시도: '{query_part}'")
                feedback_based_query = f"이전 추천에 대해 '{query_part}' 라는 피드백이 있었습니다. 이 점을 고려하여 다른 책을 추천해주세요."
                logger.info(f"피드백 기반 새 쿼리 생성: {feedback_based_query}")
                recommendation_result = await self._generate_recommendations(
                    feedback_based_query
                )
                return (
                    True,
                    "피드백 감사합니다. 말씀해주신 점을 바탕으로 다른 책을 찾아보겠습니다.\n\n"
                    + recommendation_result,
                )
            else:
                logger.warning(
                    f"처리되지 않은 후속 질문 action: {action}. 새 질문으로 간주."
                )
                return False, ""
        except Exception as e:
            logger.error(f"후속 질문 처리 중 예외 발생: {e}", exc_info=True)
            return True, "후속 질문 처리 중 오류가 발생했습니다. 다시 시도해 주세요."

    async def _handle_comparison(self, query_part: str, isbn_list: List[str]) -> str:
        logger.info(
            f"도서 비교 시작. ISBN 목록: {isbn_list}, 비교 관점: '{query_part}'"
        )
        comparison_docs_info = []
        for isbn in isbn_list:
            doc = next(
                (
                    d
                    for d in self.last_recommendations
                    if str(d.metadata.get("ISBN", "")).strip() == str(isbn).strip()
                ),
                None,
            )
            if doc:
                metadata = doc.metadata
                summary = await self._summarize_chunk_with_llm(doc.page_content[:2000])
                comparison_docs_info.append(
                    {
                        "title": metadata.get("title", "제목 없음"),
                        "author": metadata.get("author", "저자 없음"),
                        "summary": summary,
                        "category": metadata.get("category", "분류 없음"),
                        "page": metadata.get("page", "페이지 수 없음"),
                        "price": metadata.get("price", "가격 정보 없음"),
                        "isbn": isbn,
                    }
                )
            else:
                logger.error(
                    f"비교 오류: 유효성 검사 후에도 ISBN '{isbn}' 문서를 last_recommendations에서 찾지 못함."
                )
        if len(comparison_docs_info) < 2:
            logger.warning(
                f"비교 가능한 문서가 2개 미만입니다 (찾은 문서 수: {len(comparison_docs_info)})."
            )
            return "비교할 책 정보를 충분히 찾지 못했습니다."
        comparison_prompt_template = """
        다음은 사용자가 비교를 요청한 책들의 정보입니다. 사용자의 비교 요청 관점인 "{{ query }}"에 특히 초점을 맞춰 이 책들을 명확하게 비교 설명해주세요. 각 책의 주요 특징, 장르, 내용 스타일, 난이도, 분량, 가격 등 관련 정보를 활용하고, 어떤 독자에게 더 적합할지 등을 포함하여 답변하는 것이 좋습니다. 자연스러운 문장으로 설명해주세요.
        [비교 대상 책 정보]
        {% for doc in summaries %}
        --- 책 {{ loop.index }} (ISBN: {{ doc.isbn }}) ---
        제목: {{ doc.title }}
        저자: {{ doc.author }}
        분류: {{ doc.category }}
        페이지 수: {{ doc.page }}
        가격: {{ doc.price }} 원
        요약: {{ doc.summary }}
        {% endfor %}
        [사용자 비교 요청 관점/질문]
        "{{ query }}"
        [비교 설명]
        """
        try:
            comp_template = PromptTemplate(
                template=comparison_prompt_template,
                input_variables=["summaries", "query"],
                template_format="jinja2",
            )
            rendered_prompt = comp_template.render(
                summaries=comparison_docs_info, query=query_part
            )
            logger.debug(
                f"도서 비교 프롬프트 생성 완료 (일부):\n{rendered_prompt[:500]}..."
            )
            comparison_result = await async_invoke_llm(
                rendered_prompt, "도서 비교 설명 생성"
            )
            if (
                not comparison_result
                or len(comparison_result) < 20
                or "죄송" in comparison_result
                or "모르겠" in comparison_result
            ):
                logger.warning("LLM 기반 도서 비교 설명 생성 실패 또는 결과 부적절.")
                return "죄송합니다, 요청하신 책들을 비교 설명하는 데 어려움이 있습니다."
            logger.info("도서 비교 설명 생성 완료.")
            return comparison_result
        except Exception as e:
            logger.error(f"도서 비교 설명 생성 중 예외 발생: {e}", exc_info=True)
            return "죄송합니다, 책들을 비교하는 중 오류가 발생했습니다."

    async def process_query(
        self, user_query: str, force_recommendation: bool = False
    ) -> str:
        logger.info(f"=== 새로운 사용자 쿼리 처리 시작: '{user_query}' ===")
        self.user_history.append(f"사용자: {user_query}")
        await self.update_preferences_from_input(user_query)
        is_potential_followup = self.last_action == "추천" and self.last_recommendations
        if is_potential_followup:
            logger.info("이전 추천에 대한 후속 질문 가능성 확인 중...")
            handled, followup_output = await self.handle_followup_query(user_query)
            if handled:
                logger.info("후속 질문 처리 완료.")
                self.llm_history.append(f"챗봇: {followup_output}")
                return followup_output
            else:
                logger.info("후속 질문이 아님. 일반 질문 처리 로직으로 진행합니다.")
        action: Optional[str] = None
        additional_question: str = ""
        if not force_recommendation:
            logger.info("행동 결정 요청 (추천 vs 추가 질문)")
            meaningful_prefs_count = sum(
                1 for k, v in self.user_preferences.items() if v and k != "title"
            )
            should_recommend_heuristically = (
                meaningful_prefs_count >= 1 or self.preference_update_count >= 2
            )
            if not should_recommend_heuristically and self.preference_update_count < 2:
                logger.info(
                    f"선호도 부족 ({meaningful_prefs_count}개 / {self.preference_update_count}번 업데이트). '추가 질문' 강제 실행."
                )
                action = "추가 질문"
                additional_question = "어떤 종류의 책을 찾으시는지 좀 더 자세히 말씀해주시겠어요? (예: 구체적인 카테고리, 특정 작가, 책의 수준/분위기 등)"
            else:
                prompt_vars = {
                    "history": "\n".join(
                        self.user_history[-5:] + self.llm_history[-5:]
                    ),
                    "query": user_query,
                    "preferences": self.preferences_text,
                    "role_instructions": self.config.get(
                        "role_instructions", general_role
                    ),
                }
                try:
                    decision_result = await async_invoke(
                        self.decision_chain, prompt_vars, "행동 결정"
                    )
                    decision_text = decision_result.get("text", "").strip()
                    action, additional_question_llm = (
                        self.robust_parse_decision_response(decision_text)
                    )
                    if action == "추가 질문" and additional_question_llm:
                        additional_question = additional_question_llm
                except Exception as e:
                    logger.error(
                        f"행동 결정 LLM 호출 실패: {e}. '추가 질문'으로 안전하게 진행.",
                        exc_info=True,
                    )
                    action = "추가 질문"
                    additional_question = "요청을 이해하는 데 어려움이 있었습니다. 어떤 책을 찾으시는지 다시 말씀해주시겠어요?"
        else:
            logger.info("강제 추천 모드 활성화. 행동='추천'")
            action = "추천"
        response = ""
        if action == "추가 질문":
            logger.info("행동: 추가 질문")
            self.last_action = "추가 질문"
            if additional_question:
                try:
                    add_q_emb = await asyncio.to_thread(
                        self.embeddings.embed_query, additional_question
                    )
                    if is_similar_question(
                        add_q_emb,
                        previous_additional_question_embeddings,
                        threshold=0.90,
                    ):
                        logger.warning(
                            "이전과 매우 유사한 추가 질문 생성됨. 추천 강제 시도."
                        )
                        return await self.process_query(
                            user_query, force_recommendation=True
                        )
                    else:
                        previous_additional_question_embeddings.append(add_q_emb)
                        if len(previous_additional_question_embeddings) > 5:
                            previous_additional_question_embeddings.pop(0)
                        response = additional_question
                except Exception as e:
                    logger.error(f"추가 질문 임베딩 또는 유사도 비교 중 오류: {e}")
                    response = additional_question
            else:
                logger.warning("추가 질문 행동 결정되었으나 질문 내용 없음. 추천 시도.")
                action = "추천"
        if action == "추천":
            logger.info("행동: 추천")
            self.last_action = "추천"
            final_query = await self.get_final_query(user_query)
            if not final_query:
                logger.error("최종 검색 쿼리 생성 실패. 추천 불가.")
                self.last_action = None
                response = "죄송합니다, 검색어를 만드는 데 실패했습니다. 다시 질문해 주시겠어요?"
            else:
                recommendation_result = await self._generate_recommendations(
                    final_query
                )
                response = recommendation_result
                if (
                    not self.last_recommendations
                    and "죄송합니다" not in response
                    and "찾지 못했습니다" not in response
                ):
                    logger.warning(
                        "추천 생성 과정 완료 후 self.last_recommendations가 비어있으나, 응답은 성공 메시지 형태임."
                    )
                    self.last_action = None
        if response:
            self.llm_history.append(f"챗봇: {response}")
            logger.info(
                f"챗봇 응답 생성 완료 (Action: {self.last_action}). 응답 일부: {response[:200]}..."
            )
            return response
        else:
            logger.error(f"최종 응답 생성 실패 (Action: {action}).")
            self.last_action = None
            fallback_msg = (
                "죄송합니다, 요청을 처리하는 중 문제가 발생했습니다. 다시 시도해주세요."
            )
            self.llm_history.append(f"챗봇: {fallback_msg}")
            return fallback_msg

    async def interactive_multi_turn_qa(self):
        persona_greetings = {
            "Literature": "안녕하세요~ 감성적인 문학 도서 추천 챗봇입니다. 어떤 책을 찾으시나요? 당신의 이야기나 감정을 들려주셔도 좋아요.",
            "Science": "안녕하십니까. 과학/기술 도서 전문 챗봇입니다. 관심 분야, 알고 계신 내용, 찾으시는 정보의 깊이 등을 알려주시면 더 정확한 추천을 드릴 수 있습니다.",
            "General": "안녕하세요! 도서 추천 챗봇입니다. 어떤 종류의 책을 찾으시는지 편하게 말씀해주세요.",
        }
        persona = self.config.get("persona", "General")
        greeting = persona_greetings.get(persona, persona_greetings["General"])
        self.llm_history.append(f"챗봇: {greeting}")
        print("-" * 70)
        print(f"[{self.config.get('persona', '챗봇')}] {greeting}")
        print("-" * 70)
        turn_count = 0
        while True:
            turn_count += 1
            print(f"\n--- Turn {turn_count} ---")
            logger.info(f"--- Turn {turn_count} 시작 ---")
            logger.debug(
                f"Turn 시작 | last_action: {self.last_action} | last_recs: {len(self.last_recommendations)}"
            )
            try:
                user_query = input("[사용자] ").strip()
            except (KeyboardInterrupt, EOFError):
                print("\n[대화 종료]")
                break
            if user_query.lower() in ["quit", "exit", "종료", "그만"]:
                print("\n[대화 종료]")
                break
            if not user_query:
                continue
            try:
                final_answer = await self.process_query(user_query)
            except Exception as e:
                logger.critical(
                    f"대화 처리 중 치명적 오류 발생 (Turn {turn_count}): {e}",
                    exc_info=True,
                )
                final_answer = "죄송합니다. 요청을 처리하는 중에 예상치 못한 오류가 발생했습니다. 잠시 후 다시 시도해주세요."
            print("-" * 70)
            print(f"[{self.config.get('persona', '챗봇')}]\n{final_answer}")
            print("-" * 70)

In [ ]:
class LiteratureRAGPipeline(BaseRAGPipeline):
    def __init__(self, llm, embeddings, vectorstore, es_store, retriever, documents):
        config = {
            "persona": "Literature",
            "role_instructions": literature_role,
            "pref_extraction_template": extract_pref_prompt_v2,  # literature_pref_template,
            "consolidate_pref_template": consolidate_pref_prompt,
            "decision_template": decision_prompt_template,
            "final_query_template": literature_final_query_template,
            "refine_template": literature_refine_template,
            "expansion_template": literature_expansion_template,
            "re_ranking_template": re_ranking_prompt,
            "hyde_generation_template": hyde_generation_prompt,
            "hyde_keyword_template": hyde_keyword_prompt,
            "retrieval_weights": {
                "main": 0.1,
                "author": 0.5,
                "title": 0.1,
                "category": 0.3,
            },
            "persona_info": "감성, 현재 기분, 선호하는 문학 장르 및 작가 스타일",
            "recommendation_intro_template": recommendation_intro_prompt,
            "recommendation_outro_template": recommendation_outro_prompt,
        }
        super().__init__(
            config, llm, embeddings, vectorstore, es_store, retriever, documents
        )


class ScienceRAGPipeline(BaseRAGPipeline):
    def __init__(self, llm, embeddings, vectorstore, es_store, retriever, documents):
        config = {
            "persona": "Science",
            "role_instructions": science_role,
            "pref_extraction_template": extract_pref_prompt_v2,  # science_pref_template,
            "consolidate_pref_template": consolidate_pref_prompt,
            "decision_template": decision_prompt_template,
            "final_query_template": science_final_query_template,
            "refine_template": science_refine_template,
            "expansion_template": science_expansion_template,
            "re_ranking_template": re_ranking_prompt,
            "hyde_generation_template": hyde_generation_prompt,
            "hyde_keyword_template": hyde_keyword_prompt,
            "retrieval_weights": {
                "main": 0.2,
                "author": 0.1,
                "title": 0.3,
                "category": 0.4,
            },
            "persona_info": "정확, 논리, 최신 기술 동향 및 전문 지식을 반영",
            "recommendation_intro_template": recommendation_intro_prompt,
            "recommendation_outro_template": recommendation_outro_prompt,
        }
        super().__init__(
            config, llm, embeddings, vectorstore, es_store, retriever, documents
        )


class GeneralRAGPipeline(BaseRAGPipeline):
    def __init__(self, llm, embeddings, vectorstore, es_store, retriever, documents):
        config = {
            "persona": "General",
            "role_instructions": general_role,
            "pref_extraction_template": extract_pref_prompt_v2,  # general_pref_template,
            "consolidate_pref_template": consolidate_pref_prompt,
            "decision_template": decision_prompt_template,
            "final_query_template": general_final_query_template,
            "refine_template": general_refine_template,
            "expansion_template": general_expansion_template,
            "re_ranking_template": re_ranking_prompt,
            "hyde_generation_template": hyde_generation_prompt,
            "hyde_keyword_template": hyde_keyword_prompt,
            "retrieval_weights": {
                "main": 0.25,
                "author": 0.25,
                "title": 0.25,
                "category": 0.25,
            },
            "persona_info": "친절, 균형잡힌 정보, 사용자의 선호와 분위기를 반영",
            "recommendation_intro_template": recommendation_intro_prompt,
            "recommendation_outro_template": recommendation_outro_prompt,
        }
        super().__init__(
            config, llm, embeddings, vectorstore, es_store, retriever, documents
        )

In [71]:
nest_asyncio.apply()


def main():
    previous_additional_question_embeddings.clear()
    print("페르소나를 선택해주세요:")
    print("1. 예술/문학 (감성적, 문학적 표현)")
    print("2. 과학/기술 (논리적, 정확한 정보)")
    print("3. 범용/일반 (친절, 균형잡힌 정보)")
    pipeline = None
    while pipeline is None:
        choice = input("원하는 페르소나 번호를 입력하세요 (1, 2, 3): ").strip()
        pipeline_map = {
            "1": LiteratureRAGPipeline,
            "2": ScienceRAGPipeline,
            "3": GeneralRAGPipeline,
        }
        if choice in pipeline_map:
            PipelineClass = pipeline_map[choice]
            try:
                pipeline = PipelineClass(
                    llm=llm_clova,
                    embeddings=ncp_embeddings,
                    vectorstore=vectorstore,
                    es_store=es_store,
                    retriever=merged_hybrid_retriever,
                    documents=documents,
                )
                logger.info(f"선택된 페르소나: {pipeline.config['persona']}")
                print(f"\n'{pipeline.config['persona']}' 페르소나로 대화를 시작합니다.")
                print(
                    "대화를 종료하려면 'quit', 'exit', '종료', '그만' 중 하나를 입력하세요."
                )
            except Exception as e:
                logger.critical(
                    f"파이프라인 초기화 중 치명적 오류 발생: {e}", exc_info=True
                )
                print("오류: 파이프라인을 초기화할 수 없습니다. 로그를 확인하세요.")
                return
        else:
            print("잘못된 선택입니다. 1, 2, 3 중 하나를 입력해주세요.")
    try:
        asyncio.run(pipeline.interactive_multi_turn_qa())
    except Exception as e:
        logger.critical(f"메인 실행 루프에서 치명적 오류 발생: {e}", exc_info=True)
    finally:
        try:
            if connections.get_connection_addr("default"):
                connections.disconnect("default")
                logger.info("Default Milvus 연결 해제 완료.")
        except Exception as e:
            logger.warning(f"Milvus 연결 해제 중 오류 발생 (무시 가능): {e}")
        logger.info("프로그램 종료.")


if __name__ == "__main__":
    main()

페르소나를 선택해주세요:
1. 예술/문학 (감성적, 문학적 표현)
2. 과학/기술 (논리적, 정확한 정보)
3. 범용/일반 (친절, 균형잡힌 정보)


2025-04-08 11:34:59,494 - INFO - 선택된 페르소나: Literature
2025-04-08 11:34:59,495 - INFO - --- Turn 1 시작 ---



'Literature' 페르소나로 대화를 시작합니다.
대화를 종료하려면 'quit', 'exit', '종료', '그만' 중 하나를 입력하세요.
----------------------------------------------------------------------
[Literature] 안녕하세요~ 감성적인 문학 도서 추천 챗봇입니다. 어떤 책을 찾으시나요? 당신의 이야기나 감정을 들려주셔도 좋아요.
----------------------------------------------------------------------

--- Turn 1 ---


2025-04-08 11:35:08,366 - INFO - === 새로운 사용자 쿼리 처리 시작: '해리포터 시리즈같은 소설 추천해줘.' ===
2025-04-08 11:35:08,366 - INFO - 사용자 입력에서 선호도 추출 시작: '해리포터 시리즈같은 소설 추천해줘....'
2025-04-08 11:35:09,835 - INFO - HTTP Request: POST https://clovastudio.stream.ntruss.com/testapp/v1/chat-completions/HCX-003 "HTTP/1.1 200 OK"
2025-04-08 11:35:09,839 - INFO - 선호도 업데이트됨 [category]: ['소설']
2025-04-08 11:35:09,840 - INFO - 선호도 업데이트됨 [implicit info]: ['해리포터 시리즈 같은']
2025-04-08 11:35:09,840 - INFO - 선호도 업데이트 완료. 누적 업데이트 횟수: 1
2025-04-08 11:35:09,841 - INFO - 행동 결정 요청 (추천 vs 추가 질문)
2025-04-08 11:35:15,911 - INFO - HTTP Request: POST https://clovastudio.stream.ntruss.com/testapp/v1/chat-completions/HCX-003 "HTTP/1.1 200 OK"
2025-04-08 11:35:15,913 - INFO - 행동 결정 파싱 결과: 행동='추천', 추가 질문='...'
2025-04-08 11:35:15,913 - INFO - 행동: 추천
2025-04-08 11:35:15,913 - INFO - 최종 검색 쿼리 생성 시작
2025-04-08 11:35:16,729 - INFO - HTTP Request: POST https://clovastudio.stream.ntruss.com/testapp/v1/chat-completions/HCX-003 "HTTP/1.1 200 OK"


----------------------------------------------------------------------
[Literature]
네. 감성적인 판타지 소설을 찾으시는군요. 마음을 따뜻하게 해줄 수 있는 작품들로 추천해 드리겠습니다.

1. 
표지: https://image.aladin.co.kr/product/29624/14/cover500/k702838374_1.jpg
제목: 온화한 귀족의 휴가의 권장5
저자: Misaki 지음 / Momochi 일러스트
- 추천 이유: 이 책은 판타지 세계에서 재상으로 활약하던 청년 리젤이 갑작스러운 이세계 전이 후에도 자신의 능력을 발휘하여 상급 모험가와 함께 모험을 떠나는 이야기예요. 주인공은 휴가라고 생각하며 이세계에서의 삶을 즐기고 있어요.

2. 
표지: https://image.aladin.co.kr/product/23616/15/cover500/k332638520_1.jpg
제목: 유럽의 판타지 백과사전
저자: 도현신 지음
- 추천 이유: 이 책은 유럽의 다양한 신화와 전설을 바탕으로 한 판타지 세계를 다루고 있어요. 그리스 신화부터 북유럽, 켈트, 동유럽, 핀란드 신화까지 다루면서, 세상의 시작, 신, 영웅, 마법사, 괴물, 요정 등 다양한 주제에 대한 이야기를 110가지 항목으로 정리했죠. 이 책을 통해 유럽의 판타지 세계를 깊이 이해하면, 한국형 판타지 창작에도 큰 도움이 될 거예요. 또한, 이 책은 총 7권으로 기획된 '판타지 백과사전 시리즈'의 네 번째 책으로, 세계인이 함께 즐길 수 있는 다양한 문화 콘텐츠가 만들어지길 바라는 작가의 희망이 담겨 있답니다!

3. 
표지: https://image.aladin.co.kr/product/27031/26/cover500/k022730629_1.jpg
제목: 삶이 길을 묻거든
저자: 문규연 지음
- 추천 이유: 이 책은 삶의 다양한 고민과 감정, 그리고 인생 여정에서 마주치는 여러 문제들을 시로 표현한 작품이에요. 작가가 어린 시절의 

2025-04-08 11:35:47,792 - INFO - --- Turn 3 시작 ---



--- Turn 3 ---


2025-04-08 11:35:58,059 - INFO - Default Milvus 연결 해제 완료.
2025-04-08 11:35:58,059 - INFO - 프로그램 종료.



[대화 종료]
